In [1]:
# =============================================================================
# EJEMPLO DE USO MEJORADO DE CALIBRATION_NETWORK
# =============================================================================
# Este notebook demuestra las nuevas funcionalidades implementadas en 
# calibration_network.py con mejor integración de configuración y consistencia
# con las clases Set y Run.

import sys
import os
import pandas as pd

# Add the project directory to the Python path
project_path = os.path.abspath("../../")
sys.path.append(project_path)
# Add current directory to path
current_dir = os.path.abspath(".")
sys.path.append(current_dir)

# Add src directory to path
src_dir = os.path.abspath("../src")
sys.path.append(src_dir)

print("🔍 Paths configurados:")
print(f"  - Project path: {project_path}")
print(f"  - Current dir: {current_dir}")
print(f"  - Src dir: {src_dir}")

# Importar las clases mejoradas
try:
    # Force reload of modules to get latest changes
    import importlib
    if 'RTD_Calibration_VGP.src.calibration_network' in sys.modules:
        importlib.reload(sys.modules['RTD_Calibration_VGP.src.calibration_network'])
    
    from RTD_Calibration_VGP.src.calibration_network import CalibrationNetwork
    from RTD_Calibration_VGP.src.set import Set
    from RTD_Calibration_VGP.src.logfile import Logfile
    print("✅ Imports desde RTD_Calibration_VGP.src completados (módulos recargados)")
except ImportError as e:
    print(f"⚠️ Error importando desde RTD_Calibration_VGP.src: {e}")
    try:
        from calibration_network import CalibrationNetwork
        from set import Set
        from logfile import Logfile
        print("✅ Imports locales completados")
    except ImportError as e2:
        print(f"❌ Error importando clases: {e2}")
        raise e2

print("✅ Imports completados exitosamente")
print("📁 Directorio de trabajo:", os.getcwd())

🔍 Paths configurados:
  - Project path: /Users/vicky/Desktop/rtd-calibration-ana
  - Current dir: /Users/vicky/Desktop/rtd-calibration-ana/RTD_Calibration_VGP/notebooks
  - Src dir: /Users/vicky/Desktop/rtd-calibration-ana/RTD_Calibration_VGP/src
✅ Imports desde RTD_Calibration_VGP.src completados (módulos recargados)
✅ Imports completados exitosamente
📁 Directorio de trabajo: /Users/vicky/Desktop/rtd-calibration-ana/RTD_Calibration_VGP/notebooks
✅ Imports desde RTD_Calibration_VGP.src completados (módulos recargados)
✅ Imports completados exitosamente
📁 Directorio de trabajo: /Users/vicky/Desktop/rtd-calibration-ana/RTD_Calibration_VGP/notebooks


# ? Árbol de Calibración (TREE)

Este notebook implementa el **análisis completo del árbol de calibración** jerárquico entre rondas (R1 → R2 → R3).

## 🎯 Objetivo

Calcular las **constantes de calibración** de todos los sensores respecto a la **referencia absoluta** (Ronda 3), propagando offsets a través de sensores "raised" que conectan las rondas.

## ⚙️ Flujo de Procesamiento

1. **Carga de datos**: Logfile + configuración (sensors.yaml)
2. **Procesamiento por set**: 
   - Agrupación de runs por `CalibSetNumber`
   - Cálculo de offsets y RMS
   - Cálculo de constantes de calibración (weighted mean)
3. **Construcción de red**: CalibrationNetwork con grafo de conexiones
4. **Offsets encadenados**: Propagación R1 → R2 → R3
5. **Constantes finales**: Respecto a referencia absoluta

## ⚠️ CRITERIO CRÍTICO de Filtrado de Runs

**IMPORTANTE**: El filtro de runs usa **EXCLUSIÓN**, no inclusión:

### ✅ Criterio Correcto (implementado en `set.py`):
```python
# EXCLUIR solo:
if selection != 'BAD' and 'pre' not in filename and 'st' not in filename and 'lar' not in filename:
    # Incluir run
```

**Incluye**:
- Runs con `Selection = NaN` (sin etiqueta) ✅
- Runs con `Selection = ''` (vacío) ✅  
- Runs con `Selection = 'GOOD'` ✅
- Cualquier valor que NO sea 'BAD' ✅

**Excluye**:
- `Selection = 'BAD'` ❌
- Filenames con `pre`, `st`, `lar` ❌

### ❌ Error Común (NO usar):
```python
# MAL: Solo incluir GOOD pierde ~34 sets válidos
if selection == 'GOOD':  # ❌ INCORRECTO
```

**Impacto**: Con criterio erróneo se pierden ~34 de 59 sets (57% de datos perdidos)

---

## 📊 Estructura de Datos

- **`sets_dict`**: `{set_id: Set}` - Objetos Set procesados
- **`constants`**: `{set_id: DataFrame}` - Constantes por set
- **`net`**: CalibrationNetwork - Grafo de conexiones
- **`sets_ronda_1/2/3`**: Clasificación por ronda

In [2]:
# =============================================================================
# 1. CONFIGURACIÓN, CARGA DE DATOS Y PROCESAMIENTO COMPLETO
# =============================================================================

print("🚀 INICIO DE LA CELDA - Si ves esto, la celda empezó a ejecutarse")
print("="*80)

# Recargar módulos automáticamente
import importlib
import sys
import os
import pandas as pd
import yaml

if 'RTD_Calibration_VGP.src.set' in sys.modules:
    importlib.reload(sys.modules['RTD_Calibration_VGP.src.set'])
    print("🔄 Módulo 'set.py' recargado")
if 'RTD_Calibration_VGP.src.calibration_network' in sys.modules:
    importlib.reload(sys.modules['RTD_Calibration_VGP.src.calibration_network'])
    print("🔄 Módulo 'calibration_network.py' recargado")

print(f"\n📁 Directorio de trabajo: {os.getcwd()}")

# ------------------------------------------------------------------
# DEFINIR FUNCIONES AUXILIARES (ANTES DE USARLAS)
# ------------------------------------------------------------------
def get_set_round(set_id, config):
    """Obtiene la ronda de un set desde la configuración
    
    Soporta diferentes tipos de claves en el YAML (int, float, str)
    y retorna el valor de 'round' para el set especificado.
    """
    if not config or 'sensors' not in config:
        return None
    
    sets_data = config['sensors'].get('sets', {})
    
    # Intentar con diferentes tipos de clave
    for key_type in [int, float, str]:
        try:
            key = key_type(set_id)
            if key in sets_data:
                return sets_data[key].get('round')
        except (ValueError, TypeError):
            continue
    
    return None

# ------------------------------------------------------------------
# CARGAR LOGFILE
# ------------------------------------------------------------------
print("\n" + "="*80)
print("📂 CARGANDO LOGFILE")
print("="*80)

logfile_paths = [
    "../data/LogFile.csv",
    "RTD_Calibration_VGP/data/LogFile.csv",
    "../../data/LogFile.csv"
]

logfile = None
for path in logfile_paths:
    print(f"🔍 Probando ruta: {path}")
    if os.path.exists(path):
        try:
            logfile = Logfile(path)
            print(f"✅ Logfile cargado desde {path}")
            print(f"   Registros totales: {len(logfile.log_file)}")
            break
        except Exception as e:
            print(f"⚠️ Error cargando desde {path}: {e}")
    else:
        print(f"❌ Archivo no encontrado")

if logfile is None:
    raise FileNotFoundError("No se pudo encontrar el logfile en ninguna ubicación esperada")

# ------------------------------------------------------------------
# CARGAR CONFIGURACIÓN
# ------------------------------------------------------------------
print("\n" + "="*80)
print("⚙️ CARGANDO CONFIGURACIÓN")
print("="*80)

sensors_yaml_path = "../config/config.yml"
if not os.path.exists(sensors_yaml_path):
    sensors_yaml_path = "RTD_Calibration_VGP/config/config.yml"
if not os.path.exists(sensors_yaml_path):
    sensors_yaml_path = "../../config/config.yml"
if os.path.exists(sensors_yaml_path):
    print(f"✅ Configuración encontrada: {sensors_yaml_path}")
    with open(sensors_yaml_path, 'r') as f:
        sensors_config = yaml.safe_load(f)
    
    if sensors_config and 'sensors' in sensors_config:
        num_sets_config = len(sensors_config['sensors'].get('sets', {}))
        print(f"   Sets configurados: {num_sets_config}")
else:
    print("⚠️ No se encontró archivo de configuración config.yml")
    print("   Trabajando sin configuración específica")
    sensors_config = None

# ------------------------------------------------------------------
# PROCESAR TODOS LOS SETS
# ------------------------------------------------------------------
print("\n" + "="*80)
print("🔄 PROCESAMIENTO COMPLETO DE TODOS LOS SETS")
print("="*80)

# Obtener todos los sets únicos del logfile (solo numéricos)
all_set_numbers_raw = logfile.log_file['CalibSetNumber'].unique()
all_set_numbers = []

for s in all_set_numbers_raw:
    if pd.isna(s):
        continue
    try:
        # Intentar convertir a número - esto filtra automáticamente strings como "FRAME_SET1"
        set_num = float(s)
        all_set_numbers.append(int(set_num))
    except (ValueError, TypeError):
        # Ignorar sets no numéricos (FRAME_SET*, RESIST_SET*, etc.)
        continue

all_set_numbers = sorted(set(all_set_numbers))

print(f"\n📊 Sets numéricos disponibles en logfile: {len(all_set_numbers)}")

# ------------------------------------------------------------------
# ⚡ OPTIMIZACIÓN: Limitar sets a procesar
# ------------------------------------------------------------------
# MODO PRUEBA: Procesar solo sets específicos para debugging
# CRITERIO: Exclusión de BAD/pre/st/lar (NO solo GOOD, hay runs válidos sin etiqueta explícita)
TEST_SETS = [3, 4, 49, 57]
all_set_numbers = [s for s in TEST_SETS if s in all_set_numbers]
print(f"\n⚡ MODO PRUEBA: Procesando solo {len(all_set_numbers)} sets: {all_set_numbers}")
print(f"   (Criterio: EXCLUSIÓN de Selection=BAD y pre/st/lar, no solo GOOD)")

# Para procesamiento completo, comenta las líneas anteriores y descomenta esta:
# print(f"   Sets: {all_set_numbers[:20]}{'...' if len(all_set_numbers) > 20 else ''}")

# ------------------------------------------------------------------
# VALIDAR QUE TODOS LOS SETS CONFIGURADOS ESTÁN EN EL LOGFILE
# ------------------------------------------------------------------
if sensors_config and 'sensors' in sensors_config:
    configured_sets = sorted(sensors_config['sensors'].get('sets', {}).keys())
    missing_sets = [s for s in configured_sets if s not in all_set_numbers]
    
    print(f"\n🔍 Validación de disponibilidad:")
    print(f"   Sets configurados en sensors.yaml: {len(configured_sets)}")
    print(f"   Sets disponibles en logfile: {len([s for s in configured_sets if s in all_set_numbers])}")
    
    if missing_sets:
        print(f"\n⚠️ WARNING: {len(missing_sets)} sets configurados NO encontrados en logfile:")
        print(f"   Sets faltantes: {missing_sets}")
    else:
        print(f"\n✅ Todos los sets configurados están disponibles en el logfile")

# Estructuras para almacenar resultados
sets_dict = {}
constants = {}
errors = {}
set_handler = None

processed_count = 0
skipped_count = 0

total_sets = len(all_set_numbers)
print(f"\n🔄 Procesando {total_sets} sets...")
print(f"   (Mostrará progreso para cada set)\n")

for idx, set_id_int in enumerate(all_set_numbers, start=1):
    print(f"\n   🔍 [{idx}/{total_sets}] Iniciando procesamiento de Set {set_id_int}...")
    
    try:
        # Filtrar filas para este set (buscar como string, int y float)
        # IMPORTANTE: CalibSetNumber en el CSV es tipo 'object' (string)
        set_rows = logfile.log_file[
            (logfile.log_file['CalibSetNumber'] == str(set_id_int)) |
            (logfile.log_file['CalibSetNumber'] == set_id_int) |
            (logfile.log_file['CalibSetNumber'] == float(set_id_int))
        ]
        
        if set_rows.empty:
            print(f"      ⚠️ Set {set_id_int}: Sin datos (filas vacías)")
            skipped_count += 1
            continue
        
        print(f"      → Encontradas {len(set_rows)} filas en logfile")
        
        # Crear y procesar el set
        print(f"      → Creando objeto Set...")
        s = Set(set_rows, config=sensors_config)
        
        print(f"      → Agrupando runs por set...")
        s.group_runs_by_set()
        
        # Debug: verificar cuántos runs se agruparon
        if hasattr(s, 'runs_by_set') and s.runs_by_set:
            total_runs = sum(len(runs) for runs in s.runs_by_set.values())
            print(f"      → Runs agrupados: {total_runs} en {len(s.runs_by_set)} set(s)")
        else:
            print(f"      ⚠️ No se agruparon runs")
            print(f"         Causa: group_runs_by_set() excluye Selection=BAD y filenames con pre/st/lar")
            print(f"         (Criterio correcto: EXCLUSIÓN, no solo GOOD)")
            skipped_count += 1
            continue
        
        print(f"      → Calculando offsets y RMS...")
        s.calculate_offsets_and_rms()
        
        print(f"      → Calculando constantes de calibración (weighted mean)...")
        constants_dict, errors_dict = s.calculate_weighted_mean_offsets()
        
        # Las constantes se guardan en el diccionario, extraemos la del set actual
        if constants_dict and set_id_int in constants_dict:
            s.calibration_constants = constants_dict[set_id_int]
            s.calibration_errors = errors_dict.get(set_id_int, None)
        
        # Verificar que se calcularon las constantes
        if hasattr(s, 'calibration_constants') and s.calibration_constants is not None:
            # Guardar en sets_dict (para CalibrationNetwork)
            sets_dict[set_id_int] = s
            
            # Guardar constantes y errores por separado (para compatibilidad)
            constants[float(set_id_int)] = s.calibration_constants
            
            if hasattr(s, 'calibration_errors') and s.calibration_errors is not None:
                errors[float(set_id_int)] = s.calibration_errors
            else:
                errors[float(set_id_int)] = pd.DataFrame(index=s.calibration_constants.index)
            
            # Guardar primer set como referencia
            if set_handler is None:
                set_handler = s
            
            n_sensors = len(s.calibration_constants)
            processed_count += 1
            
            progress_pct = (idx / total_sets) * 100
            print(f"   ✅ [{idx}/{total_sets} - {progress_pct:.1f}%] Set {set_id_int}: {n_sensors} sensores procesados correctamente")
        else:
            print(f"      ⚠️ Set {set_id_int}: No se calcularon constantes")
            skipped_count += 1
            
    except Exception as e:
        skipped_count += 1
        print(f"   ❌ Set {set_id_int}: ERROR - {str(e)[:120]}")
        import traceback
        print(f"      Detalles: {traceback.format_exc()[:500]}")

print(f"\n📊 RESUMEN:")
print(f"   ✅ Procesados: {processed_count} sets")
print(f"   ⚠️ Omitidos: {skipped_count} sets")

if not sets_dict:
    raise RuntimeError("No se pudo procesar ningún set. Verifica el logfile y los archivos de temperatura.")

# ------------------------------------------------------------------
# DEFINIR FUNCIÓN AUXILIAR
# ------------------------------------------------------------------
def get_set_round(set_id, config):
    """Obtiene la ronda de un set desde la configuración"""
    if not config or 'sensors' not in config:
        return None
    
    sets_data = config['sensors'].get('sets', {})
    
    for key_type in [int, float, str]:
        try:
            key = key_type(set_id)
            if key in sets_data:
                return sets_data[key].get('round')
        except (ValueError, TypeError):
            continue
    
    return None

# ------------------------------------------------------------------
# CREAR RED DE CALIBRACIÓN
# ------------------------------------------------------------------
print("\n" + "="*80)
print("🌐 CREACIÓN DE RED DE CALIBRACIÓN")
print("="*80)

from RTD_Calibration_VGP.src.calibration_network import CalibrationNetwork

# Crear la red usando sets_dict
config_path = sensors_yaml_path if os.path.exists(sensors_yaml_path) else None

try:
    net = CalibrationNetwork(sets_dict, config_path=config_path)
    
    print(f"\n✅ Red creada exitosamente:")
    print(f"   Sets en la red: {len(net.sets)}")
    print(f"   Nodos en el grafo: {len(net.graph.nodes)}")
    print(f"   Conexiones (edges): {len(net.graph.edges)}")
    
    # ------------------------------------------------------------------
    # VALIDAR Y AUTO-DETECTAR SENSORES RAISED
    # ------------------------------------------------------------------
    validation_result = net.validate_and_suggest_raised_sensors(
        logfile.log_file, 
        auto_fix_missing=False,
        verbose=True
    )
    
    # Almacenar resultado para referencia
    raised_validation = validation_result
    
    if len(net.graph.edges) == 0:
        print(f"\n⚠️ WARNING: La red no tiene conexiones (grafo vacío)")
        print(f"   Esto puede indicar que faltan sensores 'raised' en la configuración")
    else:
        # Validar conectividad del grafo
        print(f"\n🔍 Validando conectividad del grafo...")
        
        # Encontrar el set de referencia (Ronda 3)
        reference_sets = [s for s in sets_dict.keys() 
                         if get_set_round(s, sensors_config) == 3]
        
        if reference_sets:
            ref_set = reference_sets[0]
            print(f"   Set de referencia (R3): {ref_set}")
            
            # Verificar cuántos sets R1 pueden conectarse al set de referencia
            r1_sets = [s for s in sets_dict.keys() 
                      if get_set_round(s, sensors_config) == 1]
            
            # Mostrar resumen de conexiones
            print(f"   Total conexiones en grafo: {len(net.graph.edges)}")
            
            connected_r1 = 0
            disconnected_r1 = []
            
            # Verificar conectividad de cada set R1 hacia la referencia
            for r1_set in r1_sets:
                try:
                    # Usar find_path_between_sets para verificar si existe un camino
                    path = net.find_path_between_sets(r1_set, ref_set)
                    if path and len(path) > 0:
                        connected_r1 += 1
                    else:
                        disconnected_r1.append(r1_set)
                except Exception:
                    # Si hay error al buscar path, considerar desconectado
                    disconnected_r1.append(r1_set)
            
            print(f"   Sets R1 conectados a referencia: {connected_r1}/{len(r1_sets)}")
            
            if disconnected_r1:
                print(f"\n⚠️ WARNING: {len(disconnected_r1)} sets R1 SIN conexión a referencia:")
                print(f"   Sets desconectados: {disconnected_r1[:10]}{'...' if len(disconnected_r1) > 10 else ''}")
            else:
                print(f"\n✅ Todos los sets R1 tienen conexión a la referencia")
        else:
            print(f"\n⚠️ WARNING: No se encontró set de referencia (Ronda 3)")
    
except Exception as e:
    print(f"\n❌ Error creando la red: {e}")
    import traceback
    traceback.print_exc()
    raise

# ------------------------------------------------------------------
# CLASIFICAR SETS POR RONDA
# ------------------------------------------------------------------
print("\n" + "="*80)
print("🔍 CLASIFICACIÓN POR RONDA")
print("="*80)

# Función get_set_round() ya está definida al inicio de la celda

sets_ronda_1 = []
sets_ronda_2 = []
sets_ronda_3 = []

for set_id in sets_dict.keys():
    round_num = get_set_round(set_id, sensors_config)
    
    if round_num == 1:
        sets_ronda_1.append(set_id)
    elif round_num == 2:
        sets_ronda_2.append(set_id)
    elif round_num == 3:
        sets_ronda_3.append(set_id)

print(f"\n📊 Sets por ronda:")
print(f"   Ronda 1: {len(sets_ronda_1)} sets → {sorted(sets_ronda_1)}")
print(f"   Ronda 2: {len(sets_ronda_2)} sets → {sorted(sets_ronda_2)}")
print(f"   Ronda 3: {len(sets_ronda_3)} sets → {sorted(sets_ronda_3)}")

# ------------------------------------------------------------------
# DEMOSTRAR MATCHING AUTOMÁTICO ENTRE RONDAS
# ------------------------------------------------------------------
print(f"\n🔍 Verificando matching automático R1 → R2 → R3:")

if sensors_config and sets_ronda_1 and sets_ronda_2:
    # Tomar primer set de ronda 1 como ejemplo
    example_r1 = sets_ronda_1[0]
    
    # Obtener sensores 'raised' del set R1
    r1_config = sensors_config['sensors']['sets'].get(example_r1, {})
    raised_sensors = r1_config.get('raised', [])
    
    if raised_sensors:
        print(f"\n   Ejemplo: Set R1={example_r1}")
        print(f"   Sensores 'raised': {raised_sensors}")
        
        # Buscar en qué sets R2 aparecen estos sensores
        matching_r2_sets = []
        
        for r2_set in sets_ronda_2:
            # Obtener los sensores del set R2
            r2_sensors = sets_dict[r2_set].calibration_constants.index.tolist()
            
            # Normalizar ambos a string para comparación
            r2_sensors_str = [str(s) for s in r2_sensors]
            
            # Verificar si algún sensor 'raised' de R1 está en R2
            for raised_sensor in raised_sensors:
                raised_str = str(raised_sensor)
                if raised_str in r2_sensors_str:
                    matching_r2_sets.append(r2_set)
                    break
        
        if matching_r2_sets:
            print(f"   ✅ Matching R2: {matching_r2_sets}")
            print(f"   → El sistema encuentra automáticamente que Set {example_r1} (R1)")
            print(f"      se conecta a Sets {matching_r2_sets} (R2) via sensores raised")
        else:
            print(f"   ⚠️ No se encontró matching con sets R2")
            print(f"      Debug: raised_sensors={raised_sensors} (tipo: {type(raised_sensors[0]).__name__})")
            print(f"      Debug: r2_sensors[:3]={r2_sensors[:3]} (tipo: {type(r2_sensors[0]).__name__})")
    else:
        print(f"   ℹ️ Set {example_r1} no tiene sensores 'raised' (posiblemente desconectado)")

selected_sets = list(sets_dict.keys())
print(f"\n✅ selected_sets: {len(selected_sets)} sets")

print("\n" + "="*80)
print("✅ CARGA Y PROCESAMIENTO COMPLETADOS")
print("="*80)

# Resumen de variables creadas
print(f"\n📝 Variables disponibles:")
print(f"   - logfile: Objeto Logfile con {len(logfile.log_file)} registros")
print(f"   - sensors_config: Configuración cargada desde YAML")
print(f"   - sets_dict: {len(sets_dict)} objetos Set procesados")
print(f"   - constants: {len(constants)} DataFrames de constantes")
print(f"   - errors: {len(errors)} DataFrames de errores")
print(f"   - set_handler: Objeto Set de referencia")
print(f"   - net: CalibrationNetwork con {len(net.sets)} sets")
print(f"   - sets_ronda_1/2/3: Clasificación por ronda")
print(f"   - selected_sets: Lista de {len(selected_sets)} sets")

print("\n🎉 FIN DE LA CELDA - Ejecución completada exitosamente")

🚀 INICIO DE LA CELDA - Si ves esto, la celda empezó a ejecutarse
🔄 Módulo 'set.py' recargado
🔄 Módulo 'calibration_network.py' recargado

📁 Directorio de trabajo: /Users/vicky/Desktop/rtd-calibration-ana/RTD_Calibration_VGP/notebooks

📂 CARGANDO LOGFILE
🔍 Probando ruta: ../data/LogFile.csv
CSV file loaded successfully from '../data/LogFile.csv'.
✅ Logfile cargado desde ../data/LogFile.csv
   Registros totales: 832

⚙️ CARGANDO CONFIGURACIÓN
✅ Configuración encontrada: ../config/config.yml
   Sets configurados: 58

🔄 PROCESAMIENTO COMPLETO DE TODOS LOS SETS

📊 Sets numéricos disponibles en logfile: 60

⚡ MODO PRUEBA: Procesando solo 4 sets: [3, 4, 49, 57]
   (Criterio: EXCLUSIÓN de Selection=BAD y pre/st/lar, no solo GOOD)

🔍 Validación de disponibilidad:
   Sets configurados en sensors.yaml: 58
   Sets disponibles en logfile: 4

⚠️ WARNING: 54 sets configurados NO encontrados en logfile:
   Sets faltantes: [5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 

11:51:39 | INFO     | Building calibration graph from configuration...
11:51:39 | INFO     | Graph built with 4 sets and 3 connections.
11:51:39 | INFO     | ✅ Tree validated: All 4 sets are connected to reference 57
11:51:39 | INFO     | Path found between 3 and 57: [3, 49, 57]
11:51:39 | INFO     | Path found between 4 and 57: [4, 49, 57]
11:51:39 | INFO     | Graph built with 4 sets and 3 connections.
11:51:39 | INFO     | ✅ Tree validated: All 4 sets are connected to reference 57
11:51:39 | INFO     | Path found between 3 and 57: [3, 49, 57]
11:51:39 | INFO     | Path found between 4 and 57: [4, 49, 57]


Calculation of constants and errors complete and saved in 'calibration_constants_and_errors.xlsx'.
   ✅ [4/4 - 100.0%] Set 57: 14 sensores procesados correctamente

📊 RESUMEN:
   ✅ Procesados: 4 sets
   ⚠️ Omitidos: 0 sets

🌐 CREACIÓN DE RED DE CALIBRACIÓN

✅ Red creada exitosamente:
   Sets en la red: 4
   Nodos en el grafo: 4
   Conexiones (edges): 3

🔍 VALIDACIÓN DE SENSORES RAISED

✅ VALIDACIÓN COMPLETA: Todos los sensores raised son correctos

🔍 Validando conectividad del grafo...
   Set de referencia (R3): 57
   Total conexiones en grafo: 3
   Sets R1 conectados a referencia: 2/2

✅ Todos los sets R1 tienen conexión a la referencia

🔍 CLASIFICACIÓN POR RONDA

📊 Sets por ronda:
   Ronda 1: 2 sets → [3, 4]
   Ronda 2: 1 sets → [49]
   Ronda 3: 1 sets → [57]

🔍 Verificando matching automático R1 → R2 → R3:

   Ejemplo: Set R1=3
   Sensores 'raised': [48203, 48479]
   ✅ Matching R2: [49]
   → El sistema encuentra automáticamente que Set 3 (R1)
      se conecta a Sets [49] (R2) via se

## ⚠️ NOTA: Funciones Definidas en el Notebook

**Funciones locales definidas en este notebook:**
- `get_set_round()` - Obtiene la ronda de un set desde la configuración

**💡 MEJORA FUTURA**: Estas funciones deberían moverse a `calibration_network.py` para:
- ✅ Reutilización: Disponibles en todo el proyecto
- ✅ Mantenimiento: Un solo lugar para actualizar
- ✅ Testing: Más fácil de testear de forma aislada
- ✅ Limpieza: El notebook se enfoca en análisis, no en implementación

**📋 Tareas pendientes:**
1. Mover `get_set_round()` → `CalibrationNetwork._get_set_round()` (o método público)
2. Actualizar referencias en el notebook para usar `net._get_set_round()` o `net.get_set_round()`
3. Eliminar definiciones locales del notebook

---

## ℹ️ Nota sobre Sensores de Referencia (Channels 13-14)

Los sensores en las posiciones 13 y 14 son **sensores de referencia externa** que:

- ✅ **Se incluyen** en `calibration_constants` (no causan problemas)
- ✅ **Se identifican automáticamente** mediante `get_reference_sensor_ids()`
- ✅ **Se excluyen** de la detección automática de sensores "raised"
- ✅ **NO se usan** para construir conexiones en el árbol de calibración

Estos sensores se repiten en múltiples sets para monitoreo, pero **NO forman parte del árbol jerárquico** de calibración (R1→R2→R3).

La validación automática a continuación verifica que estos sensores NO sean sugeridos incorrectamente como "raised".

In [3]:
# =============================================================================
# EXPORTAR SENSORES DE REFERENCIA A YAML (TODOS LOS SETS DEL LOGFILE)
# =============================================================================

print("\n" + "="*80)
print("📤 EXPORTACIÓN DE SENSORES DE REFERENCIA")
print("="*80)

# Extraer referencias para TODOS los sets en el logfile
all_refs_export = {}

# Obtener todos los sets únicos del logfile
all_sets_in_logfile = logfile.log_file['CalibSetNumber'].unique()

for set_num in all_sets_in_logfile:
    # Filtrar solo sets con números enteros válidos
    try:
        set_num_int = int(float(set_num))
        # Verificar que no sea string con caracteres no numéricos
        if str(set_num).replace('.', '').replace(',', '').replace('-', '').isalpha():
            continue
    except (ValueError, TypeError):
        # Saltar sets con nombres como FRAME_SET, RESIST_SET, etc.
        continue
    
    try:
        # Filtrar filas para este set
        set_rows_temp = logfile.log_file[
            (logfile.log_file['CalibSetNumber'] == str(set_num)) |
            (logfile.log_file['CalibSetNumber'] == set_num) |
            (logfile.log_file['CalibSetNumber'] == float(set_num))
        ]
        
        if set_rows_temp.empty:
            continue
        
        # Obtener columnas de sensores (S1, S2, ..., S20)
        sensor_cols = [col for col in set_rows_temp.columns if col.startswith('S') and col[1:].isdigit()]
        
        if len(sensor_cols) >= 2:
            # Las dos últimas columnas son las referencias
            last_col = sensor_cols[-1]
            second_last_col = sensor_cols[-2]
            
            # Extraer IDs de sensores (tomar el primer run válido)
            ref_ids = []
            for col in [second_last_col, last_col]:
                sensor_id = set_rows_temp[col].dropna().iloc[0] if not set_rows_temp[col].dropna().empty else None
                if sensor_id is not None:
                    try:
                        ref_ids.append(int(float(sensor_id)))
                    except (ValueError, TypeError):
                        pass
            
            if ref_ids:
                all_refs_export[set_num_int] = {
                    'ref_sensor_ids': sorted(ref_ids),
                    'source': 'logfile_auto_detection'
                }
                
    except Exception as e:
        # Silenciar errores para sets no numéricos
        continue

# Guardar a YAML
output_path = "../config/reference_sensors.yaml"
if not os.path.exists("../config"):
    output_path = "RTD_Calibration_VGP/config/reference_sensors.yaml"

try:
    with open(output_path, 'w') as f:
        yaml.dump(all_refs_export, f, default_flow_style=False, sort_keys=True)
    
    print(f"✅ Reference sensors exported to: {output_path}")
    print(f"   Total sets with references: {len(all_refs_export)}")
    
    # Mostrar solo los sets que están procesados (en sets_dict)
    print(f"\n📋 Sensores de referencia para sets procesados:")
    for set_id in sorted(sets_dict.keys()):
        if set_id in all_refs_export:
            ref_ids = all_refs_export[set_id]['ref_sensor_ids']
            print(f"   Set {int(set_id)}: {ref_ids}")
        else:
            print(f"   Set {int(set_id)}: ⚠️ No se encontraron referencias")
    
except Exception as e:
    print(f"❌ Error exportando referencias: {e}")
    import traceback
    traceback.print_exc()



📤 EXPORTACIÓN DE SENSORES DE REFERENCIA
✅ Reference sensors exported to: ../config/reference_sensors.yaml
   Total sets with references: 60

📋 Sensores de referencia para sets procesados:
   Set 3: [48176, 48177]
   Set 4: [48176, 48177]
   Set 49: [48177, 49262]
   Set 57: [48177, 48421]


In [4]:
# =============================================================================
# 🔍 DIAGNÓSTICO DE WARNINGS Y VALIDACIÓN
# =============================================================================
print("\n" + "="*80)
print("🔍 DIAGNÓSTICO DE VALIDACIÓN Y MATCHING")
print("="*80)

# 1. Verificar qué warnings salieron en la validación
print("\n📋 1. RESULTADO DE VALIDACIÓN:")
if 'raised_validation' in dir() and raised_validation:
    print(f"   Missing raised: {len(raised_validation.get('missing_raised', []))}")
    print(f"   Mismatched raised: {len(raised_validation.get('mismatched_raised', []))}")
    print(f"   Invalid raised: {len(raised_validation.get('invalid_raised', []))}")
    print(f"   All valid: {raised_validation.get('all_valid', False)}")
    
    # Mostrar details si hay problemas
    if not raised_validation.get('all_valid', False):
        if raised_validation.get('missing_raised'):
            print(f"\n   ⚠️ Missing raised (primeros 3):")
            for set_id, detected in raised_validation['missing_raised'][:3]:
                print(f"      Set {set_id}: detected={detected}")
        
        if raised_validation.get('mismatched_raised'):
            print(f"\n   ⚠️ Mismatched raised (primeros 3):")
            for set_id, declared, detected in raised_validation['mismatched_raised'][:3]:
                print(f"      Set {set_id}:")
                print(f"         Declared: {declared}")
                print(f"         Detected: {detected}")
        
        if raised_validation.get('invalid_raised'):
            print(f"\n   ⚠️ Invalid raised (primeros 3):")
            for set_id, invalid in raised_validation['invalid_raised'][:3]:
                print(f"      Set {set_id}: invalid={invalid}")
else:
    print("   ⚠️ No hay raised_validation disponible")

# 2. Verificar configuración de sensores YAML
print(f"\n📋 2. CONFIGURACIÓN YAML:")
if sensors_config:
    sets_in_yaml = sensors_config.get('sensors', {}).get('sets', {})
    print(f"   Sets en sensors.yaml: {len(sets_in_yaml)}")
    
    # Verificar Set 3 específicamente
    if 3 in sets_in_yaml:
        print(f"\n   Set 3:")
        print(f"      Round: {sets_in_yaml[3].get('round')}")
        print(f"      Raised: {sets_in_yaml[3].get('raised')}")
        print(f"      Discarded: {sets_in_yaml[3].get('discarded')}")
    
    # Contar por ronda
    by_round = {}
    for sid, sdata in sets_in_yaml.items():
        r = sdata.get('round')
        by_round[r] = by_round.get(r, 0) + 1
    print(f"\n   Sets por ronda: {by_round}")
else:
    print("   ⚠️ No se cargó sensors_config")

# 3. Verificar sets procesados
print(f"\n📋 3. SETS PROCESADOS:")
if 'sets_dict' in dir():
    print(f"   Total sets en sets_dict: {len(sets_dict)}")
    
    # Clasificar por ronda
    by_round_processed = {}
    for sid in sets_dict.keys():
        r = get_set_round(sid, sensors_config)
        by_round_processed[r] = by_round_processed.get(r, 0) + 1
    print(f"   Por ronda: {by_round_processed}")
    
    # Verificar Set 3
    if 3 in sets_dict or 3.0 in sets_dict:
        s3_key = 3 if 3 in sets_dict else 3.0
        s3 = sets_dict[s3_key]
        if hasattr(s3, 'calibration_constants') and s3.calibration_constants is not None:
            sensors_s3 = list(s3.calibration_constants.index)
            print(f"\n   Set 3 - Sensores: {sensors_s3}")
else:
    print("   ⚠️ No hay sets_dict disponible")

print("\n" + "="*80)


🔍 DIAGNÓSTICO DE VALIDACIÓN Y MATCHING

📋 1. RESULTADO DE VALIDACIÓN:
   Missing raised: 0
   Mismatched raised: 0
   Invalid raised: 0
   All valid: True

📋 2. CONFIGURACIÓN YAML:
   Sets en sensors.yaml: 58

   Set 3:
      Round: 1
      Raised: [48203, 48479]
      Discarded: [48205, 48478]

   Sets por ronda: {1: 47, 2: 7, 3: 2, 'Refs': 1, 4: 1}

📋 3. SETS PROCESADOS:
   Total sets en sets_dict: 4
   Por ronda: {1: 2, 2: 1, 3: 1}

   Set 3 - Sensores: ['48060', '48061', '48062', '48063', '48202', '48203', '48204', '48205', '48476', '48477', '48478', '48479', '48176', '48177']



In [5]:
# =============================================================================
# 🎯 PROCESAMIENTO DE SETS ESPECÍFICOS: 3, 4, 49 y Ronda 3
# =============================================================================
print("\n" + "="*80)
print("🎯 PROCESAMIENTO DE SETS ESPECÍFICOS: 3, 4, 49 y Ronda 3")
print("="*80)

# Definir los sets específicos que queremos procesar
sets_especificos = [3, 4, 49]
print(f"📋 Sets específicos a procesar: {sets_especificos}")

# Verificar qué sets están disponibles en la red
sets_disponibles = list(net.sets.keys()) if hasattr(net, 'sets') else []
print(f"📊 Sets disponibles en la red: {sets_disponibles}")

# Filtrar solo los sets que están disponibles
sets_a_procesar = [s for s in sets_especificos if s in sets_disponibles]
print(f"✅ Sets a procesar (disponibles): {sets_a_procesar}")

# Añadir el set de Ronda 3 si existe
try:
    set_ronda_3 = net.get_reference_set()
    
    if set_ronda_3 is None:
        print("   ⚠️ No se encontró set de Ronda 3")
    else:
        print(f"   ✅ Set de Ronda 3 encontrado: {set_ronda_3}")
        
        # Verificar si este set está disponible
        if set_ronda_3 in sets_disponibles:
            print(f"   ✅ Set de Ronda 3 ({set_ronda_3}) está disponible en la red")
            sets_a_procesar.append(set_ronda_3)
        else:
            print(f"   ⚠️ Set de Ronda 3 ({set_ronda_3}) NO está disponible en la red")
        
except Exception as e:
    print(f"   ⚠️ Error buscando set de Ronda 3: {e}")
    import traceback
    traceback.print_exc()

# Eliminar duplicados y ordenar
sets_a_procesar = sorted(list(set(sets_a_procesar)))
print(f"\n📋 LISTA FINAL DE SETS A PROCESAR: {sets_a_procesar}")

# Mostrar información detallada de cada set
print(f"\n📊 INFORMACIÓN DETALLADA DE SETS:")
for set_id in sets_a_procesar:
    try:
        # Obtener ronda del set
        round_num = net._get_set_round(set_id)
        
        print(f"\n   Set {set_id}:")
        print(f"      - Ronda: {round_num}")
        
        # Verificar si tiene calibration_constants
        if set_id in net.sets:
            set_obj = net.sets[set_id]
            if hasattr(set_obj, 'calibration_constants') and set_obj.calibration_constants is not None:
                n_sensors = len(set_obj.calibration_constants.index)
                print(f"      - Número de sensores: {n_sensors}")
                
                # Mostrar primeros sensores
                sensors_list = list(set_obj.calibration_constants.index)
                if len(sensors_list) <= 5:
                    print(f"      - Sensores: {sensors_list}")
                else:
                    print(f"      - Primeros sensores: {sensors_list[:3]} ... {sensors_list[-1]}")
            else:
                print(f"      - ⚠️ No tiene calibration_constants")
        
    except Exception as e:
        print(f"   Set {set_id}: Error obteniendo información - {e}")

print("\n" + "="*80)


🎯 PROCESAMIENTO DE SETS ESPECÍFICOS: 3, 4, 49 y Ronda 3
📋 Sets específicos a procesar: [3, 4, 49]
📊 Sets disponibles en la red: [3, 4, 49, 57]
✅ Sets a procesar (disponibles): [3, 4, 49]
   ✅ Set de Ronda 3 encontrado: 57
   ✅ Set de Ronda 3 (57) está disponible en la red

📋 LISTA FINAL DE SETS A PROCESAR: [3, 4, 49, 57]

📊 INFORMACIÓN DETALLADA DE SETS:

   Set 3:
      - Ronda: 1
      - Número de sensores: 14
      - Primeros sensores: ['48060', '48061', '48062'] ... 48177

   Set 4:
      - Ronda: 1
      - Número de sensores: 14
      - Primeros sensores: ['48480', '48481', '48482'] ... 48177

   Set 49:
      - Ronda: 2
      - Número de sensores: 14
      - Primeros sensores: ['48203', '48479', '48484'] ... 49262

   Set 57:
      - Ronda: 3
      - Número de sensores: 14
      - Primeros sensores: ['48484', '48747', '48869'] ... 48421



# ═══════════════════════════════════════════════════════════════════════════
# 🔍 SECCIÓN OPCIONAL: DEBUGGING Y ANÁLISIS DE LA ESTRUCTURA DEL ÁRBOL
# ═══════════════════════════════════════════════════════════════════════════
#
# ⚠️ **PUEDES SALTAR ESTA SECCIÓN** si solo quieres calcular offsets encadenados.
#
# Esta sección (celdas 7-14) contiene análisis detallado de:
# - Clasificación de sets por ronda
# - Visualización de conexiones del grafo
# - Validaciones de estructura
# - Análisis automático completo del árbol
#
# 💡 **ÚTIL PARA**:
#    - Debugging cuando algo falla
#    - Entender cómo está construido el árbol
#    - Validar que la estructura es correcta
#
# 🚀 **PARA CALCULAR OFFSETS**: Salta directamente a la celda 15
#
# ═══════════════════════════════════════════════════════════════════════════

In [6]:
# =============================================================================
# 🌳 ESTRUCTURA DE ÁRBOL DE CALIBRACIÓN EN CASCADA
# =============================================================================
print("\n" + "="*80)
print("🌳 ESTRUCTURA DE ÁRBOL DE CALIBRACIÓN EN CASCADA")
print("="*80)

print("""
📋 CONCEPTO DEL SISTEMA DE CALIBRACIÓN:

🔹 RONDA 3 (Referencia Absoluta):
   - Sensor de referencia absoluta: Primer sensor del sensor mapping
   - Este sensor NO se calibra, es la referencia base del sistema

🔹 RONDA 2 (Sensores 'Raised'):
   - Sensores que se calibran directamente contra la referencia de Ronda 3
   - Cada sensor 'raised' tiene un offset respecto a la referencia absoluta

🔹 RONDA 1 (Sensores de Medición):
   - Sensores que se calibran contra su correspondiente sensor 'raised' de Ronda 2
   - Cada sensor tiene un offset respecto a su sensor 'raised'

🔗 CADENA DE OFFSETS:
   Ronda 1 → Ronda 2 → Ronda 3 (Referencia Absoluta)
   
   Offset Total = Offset(R1→R2) + Offset(R2→R3) + Error_Propagado

🎯 OBJETIVO:
   Calcular offsets en cadena y propagar errores correctamente
   para obtener la calibración absoluta de cualquier sensor de Ronda 1
""")

# Identificar sensores por ronda
print("\n🔍 IDENTIFICANDO SENSORES POR RONDA:")
try:
    # Obtener todos los sets conocidos en la red
    sets_all = list(net.sets.keys())

    # Función robusta para determinar la ronda de un set
    def get_round_for_set(s):
        """Intentar obtener la ronda de varias formas: API del net, luego config (con normalización)."""
        # 1) Intentar usar la API interna
        try:
            r = net._get_set_round(s)
            if r is not None:
                return int(r)
        except Exception:
            pass

        # 2) Intentar buscar en net.config (si existe)
        try:
            cfg_sets = getattr(net, 'config', {})
            cfg_sets = cfg_sets.get('sensors', {}).get('sets', {}) if cfg_sets else {}
            if cfg_sets:
                for key, val in cfg_sets.items():
                    try:
                        # comparar varias representaciones
                        if str(key) == str(s) or float(key) == float(s):
                            rr = val.get('round')
                            if rr is not None:
                                return int(rr)
                    except Exception:
                        # fallback a comparación por string
                        if str(key) == str(s):
                            rr = val.get('round')
                            if rr is not None:
                                return int(rr)
        except Exception:
            pass

        # 3) Intentar coaccionar tipos (por si las claves son np.float64, etc.)
        try:
            s_float = float(s)
            for candidate in sets_all:
                try:
                    if float(candidate) == s_float:
                        r = None
                        try:
                            r = net._get_set_round(candidate)
                        except Exception:
                            pass
                        if r is not None:
                            return int(r)
                except Exception:
                    continue
        except Exception:
            pass

        return None

    # Clasificar sets por ronda usando la función robusta
    sets_round_1 = sorted([s for s in sets_all if get_round_for_set(s) == 1])
    sets_round_2 = sorted([s for s in sets_all if get_round_for_set(s) == 2])
    sets_round_3 = sorted([s for s in sets_all if get_round_for_set(s) == 3])

    print(f"📊 Sets encontrados:")
    print(f"   - Ronda 1: {sets_round_1}")
    print(f"   - Ronda 2: {sets_round_2}")
    print(f"   - Ronda 3: {sets_round_3}")
    
    # Identificar sensor de referencia absoluta
    if sets_round_3:
        ref_set = sets_round_3[0]  # Primer set de ronda 3
        
        # Obtener el primer sensor del set directamente desde calibration_constants
        ref_sensor = None
        if ref_set in net.sets:
            set_obj = net.sets[ref_set]
            if hasattr(set_obj, 'calibration_constants') and set_obj.calibration_constants is not None:
                # El primer sensor del índice es la referencia absoluta
                ref_sensor = set_obj.calibration_constants.index[0]
        
        print(f"\n🎯 SENSOR DE REFERENCIA ABSOLUTA:")
        print(f"   - Set: {ref_set}")
        print(f"   - Sensor ID: {ref_sensor}")
        print(f"   - Ronda: 3")
        if ref_sensor:
            print(f"   - Descripción: Este sensor NO se calibra, es la referencia base del sistema")
        else:
            print(f"   - ⚠️ No se pudo obtener el sensor de referencia (calibration_constants no disponible)")
    else:
        print("⚠️ No se encontraron sets de ronda 3 para referencia absoluta")
        
except Exception as e:
    print(f"⚠️ Error identificando sensores por ronda: {e}")
    import traceback
    traceback.print_exc()


🌳 ESTRUCTURA DE ÁRBOL DE CALIBRACIÓN EN CASCADA

📋 CONCEPTO DEL SISTEMA DE CALIBRACIÓN:

🔹 RONDA 3 (Referencia Absoluta):
   - Sensor de referencia absoluta: Primer sensor del sensor mapping
   - Este sensor NO se calibra, es la referencia base del sistema

🔹 RONDA 2 (Sensores 'Raised'):
   - Sensores que se calibran directamente contra la referencia de Ronda 3
   - Cada sensor 'raised' tiene un offset respecto a la referencia absoluta

🔹 RONDA 1 (Sensores de Medición):
   - Sensores que se calibran contra su correspondiente sensor 'raised' de Ronda 2
   - Cada sensor tiene un offset respecto a su sensor 'raised'

🔗 CADENA DE OFFSETS:
   Ronda 1 → Ronda 2 → Ronda 3 (Referencia Absoluta)

   Offset Total = Offset(R1→R2) + Offset(R2→R3) + Error_Propagado

🎯 OBJETIVO:
   Calcular offsets en cadena y propagar errores correctamente
   para obtener la calibración absoluta de cualquier sensor de Ronda 1


🔍 IDENTIFICANDO SENSORES POR RONDA:
📊 Sets encontrados:
   - Ronda 1: [3, 4]
   - Ronda

In [7]:
# =============================================================================
# 🌳 ESTRUCTURA DE ÁRBOL DE CALIBRACIÓN EN CASCADA
# =============================================================================
print("\n" + "="*80)
print("🌳 ESTRUCTURA DE ÁRBOL DE CALIBRACIÓN EN CASCADA")
print("="*80)

print("""
📋 CONCEPTO DEL SISTEMA DE CALIBRACIÓN:

🔹 RONDA 3 (Referencia Absoluta):
   - Sensor de referencia absoluta: Primer sensor del sensor mapping
   - Este sensor NO se calibra, es la referencia base del sistema

🔹 RONDA 2 (Sensores 'Raised'):
   - Sensores que se calibran directamente contra la referencia de Ronda 3
   - Cada sensor 'raised' tiene un offset respecto a la referencia absoluta

🔹 RONDA 1 (Sensores de Medición):
   - Sensores que se calibran contra su correspondiente sensor 'raised' de Ronda 2
   - Cada sensor tiene un offset respecto a su sensor 'raised'

🔗 CADENA DE OFFSETS:
   Ronda 1 → Ronda 2 → Ronda 3 (Referencia Absoluta)
   
   Offset_Total = Offset(R1→R2) + Offset(R2→R3)
   Error_Total = √(Error²_{R1→R2} + Error²_{R2→R3})
   
   Resultado: Offset_Total ± Error_Total

🎯 OBJETIVO:
   Calcular offsets en cadena y propagar errores correctamente
   para obtener la calibración absoluta de cualquier sensor de Ronda 1
""")

# =============================================================================
# 📌 CONFIGURACIÓN: SELECCIÓN DE SENSOR DE REFERENCIA ABSOLUTA
# =============================================================================
# PRIORIDAD DE SELECCIÓN:
# 1. Si el set tiene 'raised' en sensors.yaml → usar el primero de esos
# 2. Si no tiene 'raised' → usar REFERENCE_SENSOR_INDEX del calibration_constants
#
# ⚙️ PARA CAMBIAR EL SENSOR DE REFERENCIA (cuando no hay 'raised'):
# 1. Modificar REFERENCE_SENSOR_INDEX:
#    REFERENCE_SENSOR_INDEX = 0  # Primer sensor
#    REFERENCE_SENSOR_INDEX = 1  # Segundo sensor
#
# 2. O especificar un sensor ID directamente en el código más abajo
#
# =============================================================================

REFERENCE_SENSOR_INDEX = 0  # 👈 CAMBIAR AQUÍ: 0=primero, 1=segundo, 2=tercero...

# Identificar sensores por ronda
print("\n🔍 IDENTIFICANDO SENSORES POR RONDA:")
try:
    # Obtener sets por ronda usando los métodos de la clase
    sets_round_1 = net.get_sets_by_round(1)
    sets_round_2 = net.get_sets_by_round(2) 
    sets_round_3 = net.get_sets_by_round(3)
    
    print(f"📊 Sets encontrados:")
    print(f"   - Ronda 1: {sets_round_1}")
    print(f"   - Ronda 2: {sets_round_2}")
    print(f"   - Ronda 3: {sets_round_3}")
    
    # Identificar sensor de referencia absoluta
    if sets_round_3:
        ref_set = net.get_reference_set()
        
        ref_sensor = None
        selection_method = None
        
        if ref_set in net.sets:
            set_obj = net.sets[ref_set]
            if hasattr(set_obj, 'calibration_constants') and set_obj.calibration_constants is not None:
                
                # PRIORIDAD 1: Intentar obtener 'raised' desde sensors.yaml
                import yaml
                import os
                sensors_yaml_path = "../config/sensors.yaml"
                raised_sensors = []
                
                if os.path.exists(sensors_yaml_path):
                    with open(sensors_yaml_path, 'r') as f:
                        sensors_config = yaml.safe_load(f)
                    
                    if sensors_config and 'sensors' in sensors_config:
                        sets_data = sensors_config['sensors'].get('sets', {})
                        set_config = sets_data.get(ref_set, {})
                        raised_sensors = set_config.get('raised', [])
                
                # Si hay sensores 'raised' definidos, usar el primero
                if raised_sensors:
                    # Convertir a string para comparar con calibration_constants.index
                    raised_sensors_str = [str(s) for s in raised_sensors]
                    
                    # Buscar el primer raised que existe en calibration_constants
                    for raised_sensor in raised_sensors_str:
                        if raised_sensor in set_obj.calibration_constants.index:
                            ref_sensor = raised_sensor
                            selection_method = "raised_from_config"
                            print(f"\n🔸 Usando sensor 'raised' desde sensors.yaml")
                            break
                    
                    if ref_sensor is None:
                        print(f"\n⚠️ Sensores 'raised' del config no encontrados en calibration_constants")
                        print(f"   Raised en config: {raised_sensors}")
                        print(f"   Sensores disponibles: {list(set_obj.calibration_constants.index)[:5]}...")
                
                # PRIORIDAD 2: Si no hay 'raised' o no se encontró, usar índice configurable
                if ref_sensor is None:
                    if len(set_obj.calibration_constants.index) > REFERENCE_SENSOR_INDEX:
                        ref_sensor = set_obj.calibration_constants.index[REFERENCE_SENSOR_INDEX]
                        selection_method = "index_based"
                        print(f"\n🔸 Usando sensor por índice (REFERENCE_SENSOR_INDEX={REFERENCE_SENSOR_INDEX})")
                    else:
                        print(f"\n⚠️ REFERENCE_SENSOR_INDEX ({REFERENCE_SENSOR_INDEX}) fuera de rango")
                        print(f"   Set {ref_set} solo tiene {len(set_obj.calibration_constants.index)} sensores")
                        ref_sensor = set_obj.calibration_constants.index[0]
                        selection_method = "default_fallback"
                        print(f"   Usando primer sensor por defecto")
                
                # Mostrar todos los sensores disponibles
                all_sensors = list(set_obj.calibration_constants.index)
                print(f"\n📋 Sensores disponibles en Set {ref_set} (Ronda 3):")
                for idx, sensor in enumerate(all_sensors):
                    if sensor == ref_sensor:
                        marker = "👉 REFERENCIA"
                    elif selection_method == "raised_from_config" and str(sensor) in raised_sensors_str:
                        marker = "🔸 raised"
                    else:
                        marker = "  "
                    print(f"   {marker} [{idx}]: {sensor}")
        
        if ref_sensor is not None:
            print(f"\n🎯 SENSOR DE REFERENCIA ABSOLUTA SELECCIONADO:")
            print(f"   - Set: {ref_set}")
            print(f"   - Sensor ID: {ref_sensor}")
            print(f"   - Método de selección: {selection_method}")
            if selection_method == "raised_from_config":
                print(f"   - Fuente: sensors.yaml (lista 'raised')")
            elif selection_method == "index_based":
                print(f"   - Índice: {REFERENCE_SENSOR_INDEX}")
                print(f"   - Fuente: calibration_constants.index[{REFERENCE_SENSOR_INDEX}]")
            print(f"   - Ronda: 3")
            print(f"   - Descripción: Este sensor NO se calibra, es la referencia base del sistema")
        else:
            print("\n⚠️ No se pudo obtener el sensor de referencia absoluta")
            print("   Verificar que el set de referencia tiene calibration_constants")
    else:
        print("⚠️ No se encontraron sets de ronda 3 para referencia absoluta")
        
except Exception as e:
    print(f"⚠️ Error identificando sensores por ronda: {e}")
    import traceback
    traceback.print_exc()

# =============================================================================
# 🎯 PROCESAMIENTO DE SETS ESPECÍFICOS: 3, 4, 49 y Ronda 3
# =============================================================================
print("\n" + "="*80)
print("🎯 PROCESAMIENTO DE SETS ESPECÍFICOS: 3, 4, 49 y Ronda 3")
print("="*80)

# Definir los sets específicos que queremos procesar
sets_especificos = [3, 4, 49]
print(f"📋 Sets específicos a procesar: {sets_especificos}")

# Verificar qué sets están disponibles en la red
sets_disponibles = list(net.sets.keys()) if hasattr(net, 'sets') else []
print(f"📊 Sets disponibles en la red: {sets_disponibles}")

# Filtrar solo los sets que están disponibles
sets_a_procesar = [s for s in sets_especificos if s in sets_disponibles]
print(f"✅ Sets a procesar (disponibles): {sets_a_procesar}")

# Añadir el set de Ronda 3 si existe
try:
    set_ronda_3 = net.get_reference_set()
    
    if set_ronda_3 is None:
        print("   ⚠️ No se encontró set de Ronda 3")
    else:
        print(f"   ✅ Set de Ronda 3 encontrado: {set_ronda_3}")
        
        # Verificar si este set está disponible
        if set_ronda_3 in sets_disponibles:
            print(f"   ✅ Set de Ronda 3 ({set_ronda_3}) está disponible en la red")
            sets_a_procesar.append(set_ronda_3)
        else:
            print(f"   ⚠️ Set de Ronda 3 ({set_ronda_3}) NO está disponible en la red")
        
except Exception as e:
    print(f"   ⚠️ Error buscando set de Ronda 3: {e}")
    import traceback
    traceback.print_exc()

# Eliminar duplicados y ordenar
sets_a_procesar = sorted(list(set(sets_a_procesar)))
print(f"\n📋 LISTA FINAL DE SETS A PROCESAR: {sets_a_procesar}")

# Mostrar información detallada de cada set
print(f"\n📊 INFORMACIÓN DETALLADA DE SETS:")
for set_id in sets_a_procesar:
    try:
        # Obtener ronda del set
        round_num = net._get_set_round(set_id)
        
        print(f"\n   Set {set_id}:")
        print(f"      - Ronda: {round_num}")
        
        # Verificar si tiene calibration_constants
        if set_id in net.sets:
            set_obj = net.sets[set_id]
            if hasattr(set_obj, 'calibration_constants') and set_obj.calibration_constants is not None:
                n_sensors = len(set_obj.calibration_constants.index)
                print(f"      - Número de sensores: {n_sensors}")
                
                # Mostrar primeros sensores
                sensors_list = list(set_obj.calibration_constants.index)
                if len(sensors_list) <= 5:
                    print(f"      - Sensores: {sensors_list}")
                else:
                    print(f"      - Primeros sensores: {sensors_list[:3]} ... {sensors_list[-1]}")
            else:
                print(f"      - ⚠️ No tiene calibration_constants")
        
    except Exception as e:
        print(f"   Set {set_id}: Error obteniendo información - {e}")

print("\n" + "="*80)


🌳 ESTRUCTURA DE ÁRBOL DE CALIBRACIÓN EN CASCADA

📋 CONCEPTO DEL SISTEMA DE CALIBRACIÓN:

🔹 RONDA 3 (Referencia Absoluta):
   - Sensor de referencia absoluta: Primer sensor del sensor mapping
   - Este sensor NO se calibra, es la referencia base del sistema

🔹 RONDA 2 (Sensores 'Raised'):
   - Sensores que se calibran directamente contra la referencia de Ronda 3
   - Cada sensor 'raised' tiene un offset respecto a la referencia absoluta

🔹 RONDA 1 (Sensores de Medición):
   - Sensores que se calibran contra su correspondiente sensor 'raised' de Ronda 2
   - Cada sensor tiene un offset respecto a su sensor 'raised'

🔗 CADENA DE OFFSETS:
   Ronda 1 → Ronda 2 → Ronda 3 (Referencia Absoluta)

   Offset_Total = Offset(R1→R2) + Offset(R2→R3)
   Error_Total = √(Error²_{R1→R2} + Error²_{R2→R3})

   Resultado: Offset_Total ± Error_Total

🎯 OBJETIVO:
   Calcular offsets en cadena y propagar errores correctamente
   para obtener la calibración absoluta de cualquier sensor de Ronda 1


🔍 IDENTIFIC

In [8]:
# =============================================================================
# 🔗 EJEMPLO: CONSTRUIR CADENA DE CALIBRACIÓN AUTOMÁTICAMENTE
# =============================================================================
print("\n" + "="*80)
print("🔗 EJEMPLO: CONSTRUCCIÓN DE CADENA DE CALIBRACIÓN")
print("="*80)

print("""
📋 CONCEPTO:
La cadena de calibración traza el camino desde un sensor de medición (Ronda 1)
hasta el sensor de referencia absoluta (Ronda máxima), siguiendo los sensores
'raised' que sirven de puente entre rondas.

🔗 MÉTODO: CalibrationNetwork.build_calibration_chain()
   - Entrada: sensor_id (de R1), logfile DataFrame
   - Salida: Lista de (sensor_id, set_id, round_num)
   - Automático: sigue sensores 'raised' desde sensors.yaml
""")

# Ejemplo: Construir cadena para sensor 48061 (2° sensor del Set 3, NO raised)
print("\n🧪 EJEMPLO: Construir cadena para sensor 48061 (Set 3, Ronda 1) - NO raised")
print("   Mapping Set 3: 48060, 48061, 48062, 48063, 48202, 48203, ...")
print("   Sensores raised del Set 3: 48203, 48479")
print("   ✅ 48061 es el 2° sensor y NO es raised")
chain_example = net.build_calibration_chain(
    sensor_id=48061,
    logfile_df=logfile.log_file,
    verbose=True
)

print("\n" + "="*80)



🔗 EJEMPLO: CONSTRUCCIÓN DE CADENA DE CALIBRACIÓN

📋 CONCEPTO:
La cadena de calibración traza el camino desde un sensor de medición (Ronda 1)
hasta el sensor de referencia absoluta (Ronda máxima), siguiendo los sensores
'raised' que sirven de puente entre rondas.

🔗 MÉTODO: CalibrationNetwork.build_calibration_chain()
   - Entrada: sensor_id (de R1), logfile DataFrame
   - Salida: Lista de (sensor_id, set_id, round_num)
   - Automático: sigue sensores 'raised' desde sensors.yaml


🧪 EJEMPLO: Construir cadena para sensor 48061 (Set 3, Ronda 1) - NO raised
   Mapping Set 3: 48060, 48061, 48062, 48063, 48202, 48203, ...
   Sensores raised del Set 3: 48203, 48479
   ✅ 48061 es el 2° sensor y NO es raised

🔗 Construyendo cadena de calibración para sensor 48061
   ✅ Sensor 48061 encontrado en Set 3 (Ronda 1)
   🔸 Sensor raised: 48203 → buscando en Ronda 2
   ✅ Sensor raised 48203 encontrado en Set 49 (Ronda 2)
   🔸 Sensor raised: 48484 → buscando en Ronda 3
   ✅ Sensor raised 48484 encontrad

In [9]:
# =============================================================================
# 🎯 CALCULAR OFFSET TOTAL USANDO LA CADENA AUTOMÁTICA
# =============================================================================
print("\n" + "="*80)
print("🎯 CÁLCULO DE OFFSET TOTAL CON CADENA AUTOMÁTICA")
print("="*80)

print("""
📋 CONCEPTO:
Calcula el offset total desde el sensor de R1 hasta la referencia absoluta,
acumulando offsets paso a paso y propagando errores cuadráticamente.

🔗 MÉTODO: CalibrationNetwork.calculate_offset_from_chain()
   - Entrada: cadena generada por build_calibration_chain()
   - Salida: (offset_total, error_total, detalles)
   - Usa: compute_offset_between() que maneja paths en el grafo
""")

# Calcular offset para la cadena del ejemplo anterior
if chain_example:
    print("\n🧪 EJEMPLO: Calcular offset total para la cadena construida")
    offset_total, error_total, detalles = net.calculate_offset_from_chain(
        chain=chain_example,
        verbose=True
    )
    
    if offset_total is not None:
        print(f"\n✅ Cálculo completado exitosamente")
        print(f"   Cadena: {len(chain_example)} pasos")
        print(f"   Detalles disponibles en variable 'detalles'")
    else:
        print(f"\n⚠️ No se pudo calcular el offset total")
else:
    print("\n⚠️ No hay cadena de ejemplo para calcular")

print("\n" + "="*80)

11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]



🎯 CÁLCULO DE OFFSET TOTAL CON CADENA AUTOMÁTICA

📋 CONCEPTO:
Calcula el offset total desde el sensor de R1 hasta la referencia absoluta,
acumulando offsets paso a paso y propagando errores cuadráticamente.

🔗 MÉTODO: CalibrationNetwork.calculate_offset_from_chain()
   - Entrada: cadena generada por build_calibration_chain()
   - Salida: (offset_total, error_total, detalles)
   - Usa: compute_offset_between() que maneja paths en el grafo


🧪 EJEMPLO: Calcular offset total para la cadena construida

🔗 Calculando offsets para cadena de 3 pasos:

   Paso 1: Ronda 1 → Ronda 2
      Sensor 48061 (Set 3) → Sensor 48203 (Set 49)
      📊 Offset: 0.039839 ± 0.000355 mK

   Paso 2: Ronda 2 → Ronda 3
      Sensor 48203 (Set 49) → Sensor 48484 (Set 57)
      📊 Offset: -0.166408 ± 0.001274 mK

🎯 RESULTADO FINAL:
   Offset Total: -0.126569 mK
   Error Total:  0.001323 mK
   Expresión: (-0.126569 ± 0.001323) mK

✅ Cálculo completado exitosamente
   Cadena: 3 pasos
   Detalles disponibles en variable 

In [10]:
# ================================================================================
# EJEMPLO 3: CÁLCULO DE OFFSET PONDERADO CON TODOS LOS CAMINOS POSIBLES
# ================================================================================
# Ahora calculamos el offset usando TODOS los sensores raised disponibles,
# generando todos los caminos posibles y calculando una media ponderada
# con el error como peso.

print("\n" + "🌟"*40)
print("EJEMPLO: OFFSET PONDERADO CON TODOS LOS CAMINOS")
print("🌟"*80 + "\n")

# Ejemplo con el sensor 48203 (mismo que antes)
sensor_ejemplo_multipaths = 48203

print(f"Calculando offset ponderado para sensor {sensor_ejemplo_multipaths}...")
print("Este método:")
print("  1. Encuentra TODOS los sensores 'raised' disponibles")
print("  2. Construye un camino de calibración por cada sensor raised")
print("  3. Calcula offset y error para cada camino")
print("  4. Identifica el camino con menor error")
print("  5. Calcula media ponderada: w_i = 1/error_i²")
print()

# Llamar al nuevo método
offset_weighted, error_weighted, info = net.compute_weighted_offset_all_paths(
    sensor_id=sensor_ejemplo_multipaths,
    logfile_df=logfile.log_file,
    verbose=True
)

print("\n" + "="*80)
print("📋 INFORMACIÓN DETALLADA DE TODOS LOS CAMINOS")
print("="*80)

if info:
    print(f"\nNúmero total de sensores raised disponibles: {info['n_raised_sensors']}")
    print(f"Número de caminos válidos calculados: {info['n_paths']}")
    
    print(f"\n{'─'*80}")
    print("LISTADO DE CAMINOS:")
    print(f"{'─'*80}")
    
    for path in info['paths']:
        chain_str = " → ".join([f"S{s}(R{r},Set{sid})" for s, sid, r in path['chain']])
        is_best = " 🏆 MEJOR" if path['path_id'] == info['best_path']['path_id'] else ""
        print(f"\nCamino #{path['path_id']}: Raised sensor {path['raised_sensor']}{is_best}")
        print(f"  Offset: {path['offset']:.6f} ± {path['error']:.6f}")
        print(f"  Cadena: {chain_str}")
    
    print(f"\n{'─'*80}")
    print("COMPARACIÓN DE RESULTADOS:")
    print(f"{'─'*80}")
    print(f"Mejor camino individual:")
    print(f"  Offset: {info['offset_best']:.6f} ± {info['error_best']:.6f}")
    print(f"\nMedia ponderada de todos los caminos:")
    print(f"  Offset: {offset_weighted:.6f} ± {error_weighted:.6f}")
    
    mejora_error = ((info['error_best'] - error_weighted) / info['error_best']) * 100
    print(f"\nMejora en el error con media ponderada: {mejora_error:.2f}%")
    
    if error_weighted < info['error_best']:
        print("✅ La media ponderada tiene MENOR error que el mejor camino individual")
    else:
        print("ℹ️  El mejor camino individual tiene menor error que la media ponderada")
else:
    print("⚠️ No se pudo calcular información de caminos")

print("\n" + "🌟"*80)

11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]



🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
EJEMPLO: OFFSET PONDERADO CON TODOS LOS CAMINOS
🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟

Calculando offset ponderado para sensor 48203...
Este método:
  1. Encuentra TODOS los sensores 'raised' disponibles
  2. Construye un camino de calibración por cada sensor raised
  3. Calcula offset y error para cada camino
  4. Identifica el camino con menor error
  5. Calcula media ponderada: w_i = 1/error_i²


🌐 CÁLCULO DE OFFSET PONDERADO POR TODOS LOS CAMINOS POSIBLES

🔍 Sensor de partida: 48203
   ✅ Sensor encontrado en Set 3 (Ronda 1)

📋 Sensores 'raised' disponibles en Set 3: [48203, 48479]
   Total de caminos potenciales: 2

🛤️  CAMINO 1/2: Usando sensor raised 48203
   ✅ Offset: -0.166408 ± 0.001274
   Cadena: 48203 (R1) → 48203 (R2) → 48484 (R3)

🛤️  CAMINO 2/2: Usando sensor raised 48479
   ✅ Offset: -0.160556 ± 0.001099
   Cadena: 48203 (R1) → 48479 (R2) → 48484 (R3)

📊 RESUMEN DE CAMINOS CALCULADOS
   

In [11]:
# =============================================================================
# 🔍 DEBUG: Verificar qué sensores están en cada set
# =============================================================================
print("\n" + "="*80)
print("🔍 DEBUG: Verificando contenido de matrices de calibración")
print("="*80)

print("\n💡 CONCEPTO CLAVE:")
print("   - calibration_constants solo contiene offsets entre sensores DEL MISMO SET")
print("   - Para calcular offset entre sensores de diferentes sets,")
print("     debemos usar un 'sensor puente' que aparezca en ambos sets")

# Verificar Set 49
if 49 in net.sets:
    s49 = net.sets[49]
    if hasattr(s49, 'calibration_constants') and s49.calibration_constants is not None:
        print(f"\n📊 Set 49 - calibration_constants:")
        print(f"   Shape: {s49.calibration_constants.shape}")
        sensores_49 = list(s49.calibration_constants.index)
        print(f"   Sensores: {sensores_49}")
        print(f"\n   🔍 Análisis:")
        print(f"      - ¿Tiene 48203? {48203 in sensores_49 or '48203' in [str(x) for x in sensores_49]}")
        print(f"      - ¿Tiene 48484? {48484 in sensores_49 or '48484' in [str(x) for x in sensores_49]}")
        print(f"      - ¿Tiene 48747? {48747 in sensores_49 or '48747' in [str(x) for x in sensores_49]}")
        
        # Si ambos están, podemos calcular el offset directo
        if ('48203' in [str(x) for x in sensores_49] and 
            '48484' in [str(x) for x in sensores_49]):
            print(f"\n      ✅ Ambos sensores están en Set 49!")
            print(f"         Podemos calcular offset directo: 48203 → 48484")

# Verificar Set 57  
if 57 in net.sets:
    s57 = net.sets[57]
    if hasattr(s57, 'calibration_constants') and s57.calibration_constants is not None:
        print(f"\n📊 Set 57 - calibration_constants:")
        print(f"   Shape: {s57.calibration_constants.shape}")
        sensores_57 = list(s57.calibration_constants.index)
        print(f"   Sensores: {sensores_57[:8]}")
        print(f"\n   🔍 Análisis:")
        print(f"      - ¿Tiene 48203? {48203 in sensores_57 or '48203' in [str(x) for x in sensores_57]}")
        print(f"      - ¿Tiene 48484? {48484 in sensores_57 or '48484' in [str(x) for x in sensores_57]}")
        print(f"      - ¿Tiene 48747? {48747 in sensores_57 or '48747' in [str(x) for x in sensores_57]}")

print("\n💡 ESTRATEGIA CORRECTA:")
print("   Para calcular offset entre 48203 (R2) y referencia absoluta de R3:")
print("   ")
print("   Opción 1 (si 48203 y 48484 están en Set 49):")
print("      → Usar calibration_constants[48203, 48484] del Set 49")
print("   ")
print("   Opción 2 (si NO están juntos):")
print("      → Calcular offset indirecto usando un sensor puente")
print("      → Ej: offset(48203→48747) + offset(48747→48484)")

print("\n" + "="*80)


🔍 DEBUG: Verificando contenido de matrices de calibración

💡 CONCEPTO CLAVE:
   - calibration_constants solo contiene offsets entre sensores DEL MISMO SET
   - Para calcular offset entre sensores de diferentes sets,
     debemos usar un 'sensor puente' que aparezca en ambos sets

📊 Set 49 - calibration_constants:
   Shape: (14, 14)
   Sensores: ['48203', '48479', '48484', '48491', '48673', '48800', '48731', '48747', '48753', '48839', '48845', '48851', '48177', '49262']

   🔍 Análisis:
      - ¿Tiene 48203? True
      - ¿Tiene 48484? True
      - ¿Tiene 48747? True

      ✅ Ambos sensores están en Set 49!
         Podemos calcular offset directo: 48203 → 48484

📊 Set 57 - calibration_constants:
   Shape: (14, 14)
   Sensores: ['48484', '48747', '48869', '48956', '49112', '49167', '55233', '55073']

   🔍 Análisis:
      - ¿Tiene 48203? False
      - ¿Tiene 48484? True
      - ¿Tiene 48747? True

💡 ESTRATEGIA CORRECTA:
   Para calcular offset entre 48203 (R2) y referencia absoluta de R3:

# ═══════════════════════════════════════════════════════════════════════════
# 🎯 SECCIÓN PRINCIPAL: CÁLCULO DE OFFSETS ENCADENADOS
# ═══════════════════════════════════════════════════════════════════════════
#
# ✅ **ESTA ES LA SECCIÓN QUE NECESITAS** para calcular offsets entre sensores
#
# Esta sección contiene:
# - Ejemplos prácticos de cálculo de offsets
# - APIs para calcular offsets encadenados
# - Cálculo masivo de offsets para todos los sensores
#
# 📝 **DÓNDE MODIFICAR PARÁMETROS**:
#    Ver comentarios 🔧 en cada celda que indican qué modificar
#
# ═══════════════════════════════════════════════════════════════════════════

# 🌳 Análisis de la Estructura del Árbol de Calibración

Las siguientes celdas realizan un **análisis automático** de la estructura jerárquica del árbol de calibración:

## 📋 Funcionalidades Implementadas

### 1. **Carga de Configuraciones**
- Lee `config/sensors.yaml` (definición de sensores raised y rondas)
- Lee `config/tree.yaml` (reglas y estructura del árbol)

### 2. **Extracción de Sensor Mappings**
- Obtiene los **primeros 12 sensores** del mapping de cada set desde `LogFile.csv`
- Estos 12 sensores son los que se usan para construir las conexiones del árbol

### 3. **Detección Automática de Sensores Raised**
- Analiza qué sensores se repiten entre sets de diferentes rondas
- Identifica automáticamente qué sets padres (Ronda N) contribuyen sensores a sets hijos (Ronda N+1)
- Cuenta cuántos sensores aporta cada set padre

### 4. **Construcción de la Estructura del Árbol**
- Construye un diccionario `tree_structure` que mapea:
  - Cada set → sus sets padres
  - Para cada padre → qué sensores específicos aporta
  
### 5. **Visualización del Árbol**
- Imprime la estructura completa por rondas (de mayor a menor)
- Para cada set muestra:
  - Total de sensores
  - Sensores raised (desde `sensors.yaml`)
  - De qué sets hereda sensores (detección automática)
  - Validación: verifica que los sensores detectados coincidan con los raised del config

### 6. **Visualización ASCII**
- Genera un diagrama de árbol ASCII con símbolos visuales
- Muestra claramente las relaciones padre→hijo

## 🎯 Ejemplo de Uso

Una vez ejecutadas las celdas, verás algo como:

```
🔹 RONDA 3
═══════════════════════════════════════════════════════════════

📦 SET 57
   Total sensores: 12
   🔼 Sensores RAISED (sensors.yaml): []
   
   👨‍👦 COMPOSICIÓN (detectada automáticamente):
      ✅ Set 49 (R2) → 2 sensores: [48484, 48747]
      ✅ Set 50 (R2) → 2 sensores: [48869, 48956]
      ✅ Set 51 (R2) → 2 sensores: [49112, 49167]
      ✅ Set 52 (R2) → 2 sensores: [49233, 55073]
      ✅ Set 53 (R2) → 2 sensores: [55253, 55227]
      ✅ Set 54 (R2) → 2 sensores: [55233, 55221]
   
   📊 RESUMEN:
      - Total sensores heredados: 12
      - Sensores propios: 0
      - Número de parents: 6
```

## ⚙️ Configuración

Para que el análisis funcione correctamente:
- Asegúrate de que `config/sensors.yaml` esté actualizado con las definiciones de `raised` y `round`
- El `LogFile.csv` debe contener las columnas `S1`-`S12` con los sensor mappings
- Los sets deben estar en `selected_sets` y procesados con `group_runs_by_set()`

## 📝 Casos Especiales en el Árbol

### 🔄 Sets de Sensores Rescatados (46, 47, 48)
- Formados por sensores que originalmente fueron descartados en sets anteriores de Ronda 1
- Posteriormente se decidió rescatar estos sensores y formar nuevos sets
- Tienen sensores `raised` que los conectan al resto del árbol

### ⚠️ Sets Desconectados (1, 2)
- No tienen sensores `raised` definidos en `sensors.yaml`
- Están en el árbol pero desconectados de la jerarquía de calibración
- No participan en la cadena de offsets hacia la referencia absoluta

### 🔹 Sensores Propios
- **Solo los sets de Ronda 1** tienen "sensores propios" (no heredados)
- Los sets de Ronda 2 y 3 están **completamente formados** por sensores raised de rondas anteriores
- Por ejemplo: Set 57 (R3) tiene 12 sensores, todos heredados de sets de R2

### 📋 Set 12
- No existe en la numeración de sets (salto en la secuencia)

---

**Ejecuta las siguientes celdas para ver el análisis completo del árbol.**

In [12]:
# =============================================================================
# 🌳 ANÁLISIS AUTOMÁTICO DE LA ESTRUCTURA DEL ÁRBOL DE CALIBRACIÓN
# =============================================================================
print("\n" + "="*80)
print("🌳 ANÁLISIS AUTOMÁTICO DE LA ESTRUCTURA DEL ÁRBOL DE CALIBRACIÓN")
print("="*80)

import yaml
import os

# ─────────────────────────────────────────────────────────────────────────────
# 1. CARGAR CONFIGURACIONES
# ─────────────────────────────────────────────────────────────────────────────
print("\n📁 1. CARGANDO CONFIGURACIONES...")

# Cargar sensors.yaml
sensors_yaml_path = "../config/sensors.yaml"
tree_yaml_path = "../config/tree.yaml"

sensors_config = None
tree_config = None

if os.path.exists(sensors_yaml_path):
    with open(sensors_yaml_path, 'r') as f:
        sensors_config = yaml.safe_load(f)
    print(f"   ✅ Cargado: {sensors_yaml_path}")
else:
    print(f"   ⚠️ No encontrado: {sensors_yaml_path}")

if os.path.exists(tree_yaml_path):
    with open(tree_yaml_path, 'r') as f:
        tree_config = yaml.safe_load(f)
    print(f"   ✅ Cargado: {tree_yaml_path}")
else:
    print(f"   ⚠️ No encontrado: {tree_yaml_path}")

# ─────────────────────────────────────────────────────────────────────────────
# 2. EXTRAER SENSOR MAPPINGS DESDE LOS SETS PROCESADOS
# ─────────────────────────────────────────────────────────────────────────────
print("\n📊 2. EXTRAYENDO SENSOR MAPPINGS DESDE LOS SETS...")

# Usar sets_dict que contiene los objetos Set procesados
sensor_mappings = {}

# Usar sets_dict en lugar de set_handler.runs_by_set
for set_num in sets_dict.keys():
    set_obj = sets_dict[set_num]
    
    # Verificar que tiene calibration_constants
    if not hasattr(set_obj, 'calibration_constants') or set_obj.calibration_constants is None:
        print(f"   ⚠️ Set {int(set_num):2d}: No tiene calibration_constants")
        continue
    
    # Extraer sensores del calibration_constants
    sensors_list = list(set_obj.calibration_constants.index)
    
    # Tomar los primeros 12 sensores (los que se usan para el árbol)
    sensors_for_tree = sensors_list[:12]
    
    sensor_mappings[set_num] = sensors_for_tree
    print(f"   Set {int(set_num):2d}: {len(sensors_for_tree):2d} sensores → {sensors_for_tree[:4]}...")



# ─────────────────────────────────────────────────────────────────────────────
# 3. ANÁLISIS DE SENSORES RAISED (DETECTADOS AUTOMÁTICAMENTE)
# ─────────────────────────────────────────────────────────────────────────────
print("\n🔍 3. DETECTANDO SENSORES RAISED AUTOMÁTICAMENTE...")
print("   (Sensores que aparecen en sets de ronda superior)")

# Obtener rounds desde sensors.yaml
def get_round(set_num, sensors_config):
    if sensors_config and 'sensors' in sensors_config:
        sets_data = sensors_config['sensors'].get('sets', {})
        # Convertir set_num a int (las claves del YAML son integers)
        try:
            set_key = int(float(set_num))
        except:
            set_key = set_num
        
        if set_key in sets_data:
            return sets_data[set_key].get('round', None)
    return None

# Clasificar sets por ronda
sets_by_round = {}
for set_num in sensor_mappings.keys():
    round_num = get_round(set_num, sensors_config)
    
    if round_num:
        if round_num not in sets_by_round:
            sets_by_round[round_num] = []
        sets_by_round[round_num].append(set_num)

print(f"\n   📋 Sets clasificados por ronda:")
if sets_by_round:
    for round_num in sorted(sets_by_round.keys()):
        sets = sorted(sets_by_round[round_num])
        print(f"      Ronda {round_num}: {[int(s) for s in sets]}")
else:
    print(f"      ⚠️ No se clasificó ningún set por ronda")

# ─────────────────────────────────────────────────────────────────────────────
# 4. CONSTRUIR ESTRUCTURA DEL ÁRBOL
# ─────────────────────────────────────────────────────────────────────────────
print("\n🌳 4. CONSTRUYENDO ESTRUCTURA DEL ÁRBOL...")

# Función para convertir sensor IDs a string consistente
def normalize_sensor_id(sid):
    """Normaliza un sensor ID a string"""
    try:
        return str(int(float(sid)))
    except:
        return str(sid)

# Para cada set de ronda N+1, buscar qué sets de ronda N contribuyen sensores
def find_parent_sets(child_set, child_mapping, parent_sets, parent_mappings):
    """Encuentra qué sets padres contribuyen sensores al set hijo
    
    Normaliza los sensor IDs a strings para comparación consistente.
    """
    contributions = {}
    
    # Normalizar child_mapping a strings
    child_mapping_norm = [normalize_sensor_id(s) for s in child_mapping]
    
    for parent_set in parent_sets:
        parent_mapping = parent_mappings.get(parent_set, [])
        
        # Normalizar parent_mapping a strings
        parent_mapping_norm = [normalize_sensor_id(s) for s in parent_mapping]
        
        # Contar cuántos sensores del child están en el parent
        common_sensors = set(child_mapping_norm) & set(parent_mapping_norm)
        
        if len(common_sensors) > 0:
            contributions[parent_set] = {
                'sensors': sorted(list(common_sensors)),
                'count': len(common_sensors)
            }
    
    return contributions

tree_structure = {}

# Analizar cada ronda
for round_num in sorted(sets_by_round.keys()):
    if round_num == 1:
        continue  # La ronda 1 no tiene padres
    
    current_sets = sets_by_round[round_num]
    parent_round = round_num - 1
    
    if parent_round in sets_by_round:
        parent_sets = sets_by_round[parent_round]
        
        for child_set in current_sets:
            child_mapping = sensor_mappings.get(child_set, [])
            
            contributions = find_parent_sets(
                child_set, 
                child_mapping, 
                parent_sets, 
                sensor_mappings
            )
            
            tree_structure[child_set] = {
                'round': round_num,
                'total_sensors': len(child_mapping),
                'parents': contributions
            }

# ─────────────────────────────────────────────────────────────────────────────
# 5. VISUALIZACIÓN DE LA ESTRUCTURA DEL ÁRBOL
# ─────────────────────────────────────────────────────────────────────────────
print("\n" + "="*80)
print("🌳 ESTRUCTURA DEL ÁRBOL DE CALIBRACIÓN")
print("="*80)

# Función auxiliar para obtener nombre de sensores raised desde config
def get_raised_from_config(set_num, sensors_config):
    """Obtiene la lista de sensores raised definida en sensors.yaml"""
    if sensors_config and 'sensors' in sensors_config:
        sets_data = sensors_config['sensors'].get('sets', {})
        # Convertir set_num a int (las claves del YAML son integers)
        try:
            set_key = int(float(set_num))
        except:
            set_key = set_num
        
        if set_key in sets_data:
            raised = sets_data[set_key].get('raised', [])
            # Convertir a strings para comparación consistente
            return [str(int(s)) if isinstance(s, (int, float)) else str(s) for s in raised]
    return []

# Imprimir árbol por rondas
for round_num in sorted(sets_by_round.keys(), reverse=True):
    current_sets = sorted(sets_by_round[round_num])
    
    print(f"\n{'═'*80}")
    print(f"🔹 RONDA {round_num}")
    print(f"{'═'*80}")
    
    for set_num in current_sets:
        mapping = sensor_mappings.get(set_num, [])
        raised_config = get_raised_from_config(set_num, sensors_config)
        
        print(f"\n📦 SET {int(set_num)}")
        print(f"   Total sensores en mapping: {len(mapping)}")
        print(f"   Primeros 12 sensores (para árbol): {mapping}")
        
        # Sensores raised (desde config)
        if raised_config:
            print(f"   🔼 Sensores RAISED (sensors.yaml): {raised_config}")
        
        # Si tiene padres (detectados automáticamente)
        if set_num in tree_structure:
            info = tree_structure[set_num]
            parents = info['parents']
            
            if parents:
                print(f"\n   👨‍👦 COMPOSICIÓN (detectada automáticamente):")
                total_inherited = 0
                
                for parent_set in sorted(parents.keys()):
                    contrib = parents[parent_set]
                    sensors = contrib['sensors']
                    count = contrib['count']
                    total_inherited += count
                    
                    # Obtener raised del parent desde config
                    parent_raised = get_raised_from_config(parent_set, sensors_config)
                    
                    # Normalizar ambos conjuntos para comparación
                    sensors_norm = [normalize_sensor_id(s) for s in sensors]
                    parent_raised_norm = [normalize_sensor_id(s) for s in parent_raised]
                    
                    # Verificar que los sensores detectados coinciden con los raised del parent
                    match_status = "✅" if set(sensors_norm).issubset(set(parent_raised_norm)) else "⚠️"
                    
                    print(f"      {match_status} Set {int(parent_set)} (R{info['round']-1}) → {count} sensores: {sensors}")
                    
                    # Si no coincide, mostrar diferencia
                    if not set(sensors_norm).issubset(set(parent_raised_norm)):
                        missing = set(sensors_norm) - set(parent_raised_norm)
                        print(f"         ⚠️ Sensores no en raised del parent: {list(missing)}")
                
                print(f"\n   📊 RESUMEN:")
                print(f"      - Total sensores heredados: {total_inherited}")
                if round_num == 1:
                    print(f"      - Sensores propios: {len(mapping) - total_inherited}")
                print(f"      - Número de parents: {len(parents)}")
        else:
            # Set sin padres detectados (ej: Sets 1, 2, 46, 47, 48)
            if round_num == 1:
                # Verificar si tiene raised en el config
                if not raised_config:
                    print(f"\n   ⚠️ SET DESCONECTADO: No tiene sensores raised")
                    print(f"      - Razón: Sin sensores para conectar con rondas superiores")
                elif int(set_num) in [46, 47, 48]:
                    print(f"\n   🔄 SET DE SENSORES RESCATADOS:")
                    print(f"      - Formado por sensores originalmente descartados")
                    print(f"      - Sensores raised: {raised_config}")
                else:
                    print(f"\n   📊 RESUMEN:")
                    print(f"      - Sensores propios: {len(mapping)}")
                    print(f"      - Sin herencia detectada de otros sets")

print("\n" + "="*80)
print("✅ ANÁLISIS COMPLETO")
print("="*80)


🌳 ANÁLISIS AUTOMÁTICO DE LA ESTRUCTURA DEL ÁRBOL DE CALIBRACIÓN

📁 1. CARGANDO CONFIGURACIONES...
   ✅ Cargado: ../config/sensors.yaml
   ✅ Cargado: ../config/tree.yaml

📊 2. EXTRAYENDO SENSOR MAPPINGS DESDE LOS SETS...
   Set  3: 12 sensores → ['48060', '48061', '48062', '48063']...
   Set  4: 12 sensores → ['48480', '48481', '48482', '48483']...
   Set 49: 12 sensores → ['48203', '48479', '48484', '48491']...
   Set 57: 12 sensores → ['48484', '48747', '48869', '48956']...

🔍 3. DETECTANDO SENSORES RAISED AUTOMÁTICAMENTE...
   (Sensores que aparecen en sets de ronda superior)

   📋 Sets clasificados por ronda:
      Ronda 1: [3, 4]
      Ronda 2: [49]
      Ronda 3: [57]

🌳 4. CONSTRUYENDO ESTRUCTURA DEL ÁRBOL...

🌳 ESTRUCTURA DEL ÁRBOL DE CALIBRACIÓN

════════════════════════════════════════════════════════════════════════════════
🔹 RONDA 3
════════════════════════════════════════════════════════════════════════════════

📦 SET 57
   Total sensores en mapping: 12
   Primeros 12 sens

In [13]:
# =============================================================================
# 🎨 VISUALIZACIÓN EN FORMATO ÁRBOL ASCII
# =============================================================================
print("\n" + "="*80)
print("🎨 VISUALIZACIÓN EN FORMATO ÁRBOL")
print("="*80)

def print_tree_visual(sets_by_round, tree_structure, sensor_mappings, sensors_config):
    """Imprime un árbol visual ASCII de la estructura de calibración"""
    
    # Ordenar rondas de mayor a menor (empezar por la referencia)
    rounds = sorted(sets_by_round.keys(), reverse=True)
    
    for round_num in rounds:
        current_sets = sorted(sets_by_round[round_num])
        
        # Encabezado de ronda
        if round_num == 3:
            print(f"\n{'╔'+'═'*78+'╗'}")
            print(f"║ 🎯 RONDA {round_num} - REFERENCIA ABSOLUTA{' '*45}║")
            print(f"{'╚'+'═'*78+'╝'}")
        elif round_num == 2:
            print(f"\n{'┏'+'━'*78+'┓'}")
            print(f"┃ 🔼 RONDA {round_num} - SENSORES RAISED (Intermedios){' '*38}┃")
            print(f"{'┗'+'━'*78+'┛'}")
        else:
            print(f"\n{'┌'+'─'*78+'┐'}")
            print(f"│ 📊 RONDA {round_num} - SENSORES DE MEDICIÓN{' '*43}│")
            print(f"{'└'+'─'*78+'┘'}")
        
        for set_num in current_sets:
            mapping = sensor_mappings.get(set_num, [])
            raised_config = get_raised_from_config(set_num, sensors_config)
            
            # Símbolo según ronda y tipo de set
            if round_num == 3:
                symbol = "🎯"
            elif round_num == 2:
                symbol = "🔼"
            elif int(set_num) in [46, 47, 48]:
                symbol = "🔄"  # Sensores rescatados
            elif not raised_config:
                symbol = "⚠️"  # Sets desconectados (1, 2)
            else:
                symbol = "📦"
            
            print(f"\n  {symbol} SET {int(set_num)}")
            print(f"     Sensores totales: {len(mapping)}")
            
            # Indicadores especiales
            if int(set_num) in [46, 47, 48]:
                print(f"     🔄 Sensores rescatados (originalmente descartados)")
            elif round_num == 1 and not raised_config:
                print(f"     ⚠️ SET DESCONECTADO (sin sensores raised)")
            
            if raised_config:
                print(f"     Raised: {raised_config}")
            
            # Si tiene estructura de árbol
            if set_num in tree_structure:
                info = tree_structure[set_num]
                parents = info['parents']
                
                if parents:
                    print(f"     ↓ Hereda de:")
                    
                    parent_list = sorted(parents.keys())
                    for i, parent_set in enumerate(parent_list):
                        contrib = parents[parent_set]
                        count = contrib['count']
                        sensors = contrib['sensors']
                        
                        # Usar diferentes caracteres para el último elemento
                        if i == len(parent_list) - 1:
                            prefix = "     └──"
                        else:
                            prefix = "     ├──"
                        
                        print(f"{prefix} Set {int(parent_set)}: {count} sensores {sensors[:2]}...")
            elif round_num == 1:
                # Sets de Ronda 1 sin padres
                if raised_config:
                    print(f"     ✓ Sensores propios: {len(mapping)}")
                else:
                    print(f"     ⚠️ Sin conexión al árbol")
    
    print("\n" + "="*80)

# Llamar a la visualización
print_tree_visual(sets_by_round, tree_structure, sensor_mappings, sensors_config)

# ─────────────────────────────────────────────────────────────────────────────
# RESUMEN ESTADÍSTICO
# ─────────────────────────────────────────────────────────────────────────────
print("\n📊 RESUMEN ESTADÍSTICO DEL ÁRBOL")
print("="*80)

total_sets = sum(len(sets_by_round[r]) for r in sets_by_round.keys())
print(f"  Total de sets en el árbol: {total_sets}")
print(f"  Rondas detectadas: {sorted(sets_by_round.keys())}")

for round_num in sorted(sets_by_round.keys()):
    count = len(sets_by_round[round_num])
    print(f"    - Ronda {round_num}: {count} sets")

print(f"\n  Sets con estructura de árbol detectada: {len(tree_structure)}")

# Contar conexiones padre-hijo
total_connections = 0
for set_num, info in tree_structure.items():
    total_connections += len(info['parents'])

print(f"  Total de conexiones padre→hijo: {total_connections}")

# Identificar sets especiales
sets_rescatados = [46, 47, 48]
sets_desconectados = []
for set_num in sensor_mappings.keys():
    round_num = get_round(set_num, sensors_config)
    if round_num == 1:
        raised = get_raised_from_config(set_num, sensors_config)
        if not raised and int(set_num) not in sets_rescatados:
            sets_desconectados.append(int(set_num))

print(f"\n  📝 Casos especiales:")
if sets_rescatados:
    sets_rescatados_presentes = [s for s in sets_rescatados if s in [int(k) for k in sensor_mappings.keys()]]
    if sets_rescatados_presentes:
        print(f"    🔄 Sets de sensores rescatados: {sets_rescatados_presentes}")
        print(f"       (Formados por sensores originalmente descartados)")
if sets_desconectados:
    print(f"    ⚠️ Sets desconectados (sin raised): {sets_desconectados}")
    print(f"       (No están conectados al árbol de calibración)")

print("\n" + "="*80)


🎨 VISUALIZACIÓN EN FORMATO ÁRBOL

╔══════════════════════════════════════════════════════════════════════════════╗
║ 🎯 RONDA 3 - REFERENCIA ABSOLUTA                                             ║
╚══════════════════════════════════════════════════════════════════════════════╝

  🎯 SET 57
     Sensores totales: 12
     Raised: ['48484']
     ↓ Hereda de:
     └── Set 49: 2 sensores ['48484', '48747']...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🔼 RONDA 2 - SENSORES RAISED (Intermedios)                                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

  🔼 SET 49
     Sensores totales: 12
     Raised: ['48484', '48747']
     ↓ Hereda de:
     ├── Set 3: 2 sensores ['48203', '48479']...
     └── Set 4: 2 sensores ['48484', '48491']...

┌──────────────────────────────────────────────────────────────────────────────┐
│ 📊 RONDA 1 - SENSORES DE MEDICIÓN                                           │
└

In [14]:
# =============================================================================
# 🎯 SELECCIÓN DE SENSORES Y CÁLCULO DE CADENA DE OFFSETS
# =============================================================================
print("\n" + "="*80)
print("🎯 SELECCIÓN DE SENSORES Y CÁLCULO DE CADENA DE OFFSETS")
print("="*80)

# Función para calcular cadena completa de offsets
def calculate_offset_chain(net, sensor_r1, sensor_r2_raised, sensor_r3_reference):
    """
    Calcula la cadena completa de offsets desde un sensor de Ronda 1 hasta la referencia absoluta
    
    Args:
        sensor_r1: Sensor de Ronda 1 (el que queremos calibra)
        sensor_r2_raised: Sensor 'raised' de Ronda 2 correspondiente
        sensor_r3_reference: Sensor de referencia absoluta de Ronda 3
    
    Returns:
        tuple: (offset_total, error_total, detalles)
    """
    print(f"\n🔗 CALCULANDO CADENA DE OFFSETS:")
    print(f"   Sensor R1: {sensor_r1}")
    print(f"   Sensor R2: {sensor_r2_raised}")
    print(f"   Sensor R3: {sensor_r3_reference}")
    
    detalles = {}
    
    try:
        # Offset R1 → R2
        offset_r1_r2, error_r1_r2 = net.compute_offset_between(sensor_r1, sensor_r2_raised)
        detalles['r1_r2'] = {'offset': offset_r1_r2, 'error': error_r1_r2}
        print(f"   📊 Offset R1→R2: {offset_r1_r2:.6f} ± {error_r1_r2:.6f}")
        
        # Offset R2 → R3
        print(f"   🔍 Calculando offset R2→R3 entre {sensor_r2_raised} y {sensor_r3_reference}...")
        print(f"      Tipo sensor_r2_raised: {type(sensor_r2_raised)}, valor: {repr(sensor_r2_raised)}")
        print(f"      Tipo sensor_r3_reference: {type(sensor_r3_reference)}, valor: {repr(sensor_r3_reference)}")
        offset_r2_r3, error_r2_r3 = net.compute_offset_between(sensor_r2_raised, sensor_r3_reference)
        detalles['r2_r3'] = {'offset': offset_r2_r3, 'error': error_r2_r3}
        print(f"   📊 Offset R2→R3: {offset_r2_r3:.6f} ± {error_r2_r3:.6f}")
        
        # Offset total (suma de offsets en la cadena)
        offset_total = offset_r1_r2 + offset_r2_r3
        
        # Error total (propagación cuadrática de errores independientes)
        import numpy as np
        error_total = np.sqrt(error_r1_r2**2 + error_r2_r3**2)
        
        detalles['total'] = {'offset': offset_total, 'error': error_total}
        
        print(f"\n🎯 RESULTADO FINAL:")
        print(f"   Offset Total: {offset_total:.6f} mK")
        print(f"   Error Total:  {error_total:.6f} mK")
        print(f"   Expresión: ({offset_total:.6f} ± {error_total:.6f}) mK")
        
        # Resumen visual de la cadena
        print(f"\n" + "="*80)
        print(f"📊 RESUMEN VISUAL DE LA CADENA:")
        print(f"="*80)
        print(f"")
        print(f"   Set R1              Set R2              Set R3")
        print(f"  ┌─────────┐         ┌─────────┐         ┌─────────┐")
        print(f"  │ {sensor_r1:>7} │         │ {sensor_r2_raised:>7} │         │ {sensor_r3_reference:>7} │")
        print(f"  │ (target)│         │(raised) │         │  (ref)  │")
        print(f"  │         │  ────→  │         │  ────→  │         │")
        print(f"  └─────────┘         └─────────┘         └─────────┘")
        print(f"      ↓                   ↓                   ↓")
        print(f"  {offset_r1_r2:+.6f} mK      {offset_r2_r3:+.6f} mK      (referencia)")
        print(f"")
        print(f"  ➡️  Resultado: {sensor_r1} está a {offset_total:+.6f} mK de la referencia {sensor_r3_reference}")
        print(f"=" * 80)
        
        return offset_total, error_total, detalles
        
    except Exception as e:
        print(f"⚠️ Error calculando cadena de offsets: {e}")
        import traceback
        traceback.print_exc()
        return None, None, {}

# Ejemplo REAL usando los sensores del árbol detectado
print("\n🧪 EJEMPLO DE CÁLCULO CON SENSORES REALES:")
try:
    # Obtener sensores reales del árbol
    if not sets_by_round:
        raise ValueError("No hay sets clasificados por ronda. Ejecuta primero la celda 8.")
    
    # Verificar que tenemos las 3 rondas
    if 1 not in sets_by_round or 2 not in sets_by_round or 3 not in sets_by_round:
        raise ValueError(f"Faltan rondas. Disponibles: {list(sets_by_round.keys())}")
    
    # Seleccionar un set de Ronda 1 (ej: Set 3)
    set_r1 = 3.0
    mapping_r1 = sensor_mappings.get(set_r1, [])
    if not mapping_r1:
        raise ValueError(f"No hay mapping para Set {set_r1}")
    
    # Primer sensor del Set 3 (Ronda 1) que queremos calibrar
    sensor_r1_ejemplo = mapping_r1[0]  # '48060'
    
    # Obtener los sensores raised del Set 3 desde sensors.yaml
    raised_r1 = get_raised_from_config(set_r1, sensors_config)
    if not raised_r1:
        raise ValueError(f"No hay raised para Set {set_r1}")
    
    # DETECCIÓN AUTOMÁTICA del sensor R2:
    # Buscar en qué set de R2 aparece el primer sensor raised del Set 3
    sensor_r2_ejemplo = raised_r1[0]  # Sensor raised que queremos encontrar en R2
    set_r2_encontrado = None
    
    print(f"🔍 Buscando sensor raised {sensor_r2_ejemplo} (tipo: {type(sensor_r2_ejemplo).__name__}) en sets de Ronda 2...")
    print(f"   Sets R2 disponibles: {[int(s) for s in sets_by_round[2]]}")
    
    for set_r2 in sets_by_round[2]:
        mapping_r2 = sensor_mappings.get(set_r2, [])
        
        # Normalizar ambos para comparación: convertir todo a int
        mapping_r2_int = []
        for s in mapping_r2:
            try:
                mapping_r2_int.append(int(float(s)) if isinstance(s, str) else int(s))
            except (ValueError, TypeError):
                pass
        
        sensor_r2_int = int(float(sensor_r2_ejemplo)) if not isinstance(sensor_r2_ejemplo, int) else sensor_r2_ejemplo
        
        if sensor_r2_int in mapping_r2_int:
            set_r2_encontrado = set_r2
            print(f"   ✅ Encontrado en Set {int(set_r2)}")
            print(f"      Mapping R2 (primeros 6): {mapping_r2_int[:6]}")
            break
        else:
            print(f"   ❌ Set {int(set_r2)}: No contiene {sensor_r2_int} (tiene {len(mapping_r2_int)} sensores)")
    
    if not set_r2_encontrado:
        print(f"   ⚠️ No se encontró matching con sets R2")
        raise ValueError(f"Sensor raised {sensor_r2_ejemplo} no encontrado en ningún set de R2")
    
    # Obtener sensor de referencia de Ronda 3 (Set 57)
    set_r3 = 57.0
    mapping_r3 = sensor_mappings.get(set_r3, [])
    if not mapping_r3:
        raise ValueError(f"No hay mapping para Set {set_r3}")
    
    sensor_r3_ejemplo = mapping_r3[0]  # Primer sensor del Set 57 (referencia absoluta)
    
    print(f"\n📋 Sensores reales seleccionados (detección automática):")
    print(f"   R1 (Set {int(set_r1)}): {sensor_r1_ejemplo}")
    print(f"   R2 (Set {int(set_r2_encontrado)}, raised de Set {int(set_r1)}): {sensor_r2_ejemplo}")
    print(f"   R3 (Set {int(set_r3)}, referencia): {sensor_r3_ejemplo}")
    
    # Calcular cadena
    offset_total, error_total, detalles = calculate_offset_chain(
        net, sensor_r1_ejemplo, sensor_r2_ejemplo, sensor_r3_ejemplo
    )
    
    if offset_total is not None:
        print(f"\n✅ Cadena de offsets calculada exitosamente")
        print(f"   Detalles: {detalles}")
    else:
        print(f"\n⚠️ No se pudo calcular la cadena de offsets")
        
except Exception as e:
    print(f"⚠️ Error en ejemplo: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)


11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]



🎯 SELECCIÓN DE SENSORES Y CÁLCULO DE CADENA DE OFFSETS

🧪 EJEMPLO DE CÁLCULO CON SENSORES REALES:
🔍 Buscando sensor raised 48203 (tipo: str) en sets de Ronda 2...
   Sets R2 disponibles: [49]
   ✅ Encontrado en Set 49
      Mapping R2 (primeros 6): [48203, 48479, 48484, 48491, 48673, 48800]

📋 Sensores reales seleccionados (detección automática):
   R1 (Set 3): 48060
   R2 (Set 49, raised de Set 3): 48203
   R3 (Set 57, referencia): 48484

🔗 CALCULANDO CADENA DE OFFSETS:
   Sensor R1: 48060
   Sensor R2: 48203
   Sensor R3: 48484
   📊 Offset R1→R2: 0.073636 ± 0.000624
   🔍 Calculando offset R2→R3 entre 48203 y 48484...
      Tipo sensor_r2_raised: <class 'str'>, valor: '48203'
      Tipo sensor_r3_reference: <class 'str'>, valor: '48484'
   📊 Offset R2→R3: -0.166408 ± 0.001274

🎯 RESULTADO FINAL:
   Offset Total: -0.092772 mK
   Error Total:  0.001419 mK
   Expresión: (-0.092772 ± 0.001419) mK

📊 RESUMEN VISUAL DE LA CADENA:

   Set R1              Set R2              Set R3
  ┌──────

In [15]:
# =============================================================================
# 🔍 DIAGNÓSTICO: Búsqueda de Sensores Raised en Sets R2
# =============================================================================
print("\n" + "="*80)
print("🔍 DIAGNÓSTICO DE BÚSQUEDA DE RAISED EN R2")
print("="*80)

# Verificar datos para Set 3
if 'sensors_config' in dir() and 'sensor_mappings' in dir() and 'sets_by_round' in dir():
    set_r1_test = 3.0
    
    print(f"\n📋 Set 3 (Ronda 1):")
    
    # Raised desde config
    raised_test = get_raised_from_config(set_r1_test, sensors_config)
    print(f"   Raised desde sensors.yaml: {raised_test}")
    print(f"   Tipos: {[type(r).__name__ for r in raised_test]}")
    
    # Verificar sensor_mappings para sets R2
    if 2 in sets_by_round:
        print(f"\n📋 Sets de Ronda 2: {[int(s) for s in sets_by_round[2]]}")
        
        for set_r2 in sets_by_round[2][:3]:  # Primeros 3 para no saturar
            mapping = sensor_mappings.get(set_r2, [])
            print(f"\n   Set {int(set_r2)}:")
            print(f"      Total sensores: {len(mapping)}")
            print(f"      Primeros 6: {mapping[:6]}")
            print(f"      Tipos: {[type(m).__name__ for m in mapping[:3]]}")
            
            # Verificar si algún raised está presente
            for raised_sensor in raised_test:
                # Normalizar a int para comparación
                try:
                    raised_int = int(float(raised_sensor))
                    mapping_ints = []
                    for m in mapping:
                        try:
                            mapping_ints.append(int(float(m)))
                        except:
                            pass
                    
                    if raised_int in mapping_ints:
                        idx = mapping_ints.index(raised_int)
                        print(f"      ✅ Raised {raised_sensor} → ENCONTRADO (posición {idx})")
                    else:
                        print(f"      ❌ Raised {raised_sensor} → NO encontrado")
                except Exception as e:
                    print(f"      ⚠️ Error procesando {raised_sensor}: {e}")
    else:
        print("\n   ⚠️ No hay sets de Ronda 2 en sets_by_round")
else:
    print("\n   ⚠️ Variables necesarias no disponibles")

print("\n" + "="*80)


🔍 DIAGNÓSTICO DE BÚSQUEDA DE RAISED EN R2

📋 Set 3 (Ronda 1):
   Raised desde sensors.yaml: ['48203', '48479']
   Tipos: ['str', 'str']

📋 Sets de Ronda 2: [49]

   Set 49:
      Total sensores: 12
      Primeros 6: ['48203', '48479', '48484', '48491', '48673', '48800']
      Tipos: ['str', 'str', 'str']
      ✅ Raised 48203 → ENCONTRADO (posición 0)
      ✅ Raised 48479 → ENCONTRADO (posición 1)



# 🌳 Guía de Uso del Sistema de Calibración en Cascada

## 📋 Estructura del Sistema

El sistema de calibración está diseñado como un **árbol de offsets en cascada** con 3 rondas:

### 🔹 **Ronda 3 - Referencia Absoluta**
- **Sensor**: Primer sensor del sensor mapping del primer set de Ronda 3
- **Función**: Referencia base del sistema (NO se calibra)
- **Identificación**: Se obtiene automáticamente usando `net._get_reference_sensor(ref_set)`

### 🔹 **Ronda 2 - Sensores 'Raised'**
- **Sensores**: Sensores que se calibran directamente contra la referencia de Ronda 3
- **Función**: Intermediarios entre Ronda 1 y la referencia absoluta
- **Identificación**: Se obtienen de la configuración usando `net._get_reference_sensor(set_id)`

### 🔹 **Ronda 1 - Sensores de Medición**
- **Sensores**: Sensores que se calibran contra su correspondiente sensor 'raised' de Ronda 2
- **Función**: Sensores finales que queremos calibrar
- **Identificación**: Cualquier sensor de un set de Ronda 1

## 🔗 Cadena de Offsets

```
Sensor R1 → Sensor R2 (raised) → Sensor R3 (referencia absoluta)
```

**Fórmula del Offset Total:**
```
Offset_Total = Offset(R1→R2) + Offset(R2→R3)
Error_Total = √(Error²_{R1→R2} + Error²_{R2→R3})
```

Nota: Los errores se propagan cuadráticamente (suma de cuadrados bajo raíz), no se suman linealmente.

## 🎯 Cómo Usar el Sistema

### 1. **Identificar Sensores por Ronda**
```python
# Obtener sets por ronda
sets_round_1 = net.get_sets_by_round(1)
sets_round_2 = net.get_sets_by_round(2) 
sets_round_3 = net.get_sets_by_round(3)

# Identificar sensor de referencia absoluta
ref_set = sets_round_3[0]  # Primer set de ronda 3
ref_sensor = net._get_reference_sensor(ref_set)
```

### 2. **Seleccionar Sensores para Calibración**
```python
# Sensor de Ronda 1 que queremos calibrar
sensor_r1 = 48060  # Primer sensor del Set 3, por ejemplo

# El sensor R2 se DETECTA AUTOMÁTICAMENTE:
# - Se obtienen los raised del Set R1 desde sensors.yaml
# - Se busca en qué set de R2 aparece ese sensor raised
raised_r1 = get_raised_from_config(set_r1, sensors_config)
sensor_r2_raised = raised_r1[0]  # Primer sensor raised

# Buscar en qué set de R2 está este sensor
for set_r2 in sets_by_round[2]:
    if sensor_r2_raised in sensor_mappings[set_r2]:
        set_r2_encontrado = set_r2
        break

# Sensor de referencia absoluta de Ronda 3
sensor_r3_reference = sensor_mappings[57.0][0]  # Primer sensor del Set 57
```

### 3. **Calcular Cadena de Offsets**
```python
# Usar la función calculate_offset_chain()
offset_total, error_total, detalles = calculate_offset_chain(
    net, sensor_r1, sensor_r2_raised, sensor_r3_reference
)
```

## 📊 Interpretación de Resultados

- **Offset Total**: Desviación total del sensor R1 respecto a la referencia absoluta
- **Error Total**: Incertidumbre propagada a través de la cadena
- **Detalles**: Breakdown de cada offset individual en la cadena

## 🚀 Extensión Futura

El sistema está diseñado para crecer a **4 rondas** cuando sea necesario:
- Ronda 4: Nueva referencia absoluta
- Ronda 3: Sensores 'raised' intermedios
- Ronda 2: Sensores 'raised' intermedios
- Ronda 1: Sensores de medición final

La estructura modular permite agregar nuevas rondas sin modificar el código existente.


# ═══════════════════════════════════════════════════════════════════════════
# 🚀 GUÍA RÁPIDA: Cómo calcular offsets entre DOS SENSORES
# ═══════════════════════════════════════════════════════════════════════════

## 📝 Pasos para calcular el offset entre dos sensores:

### 1️⃣ **Identifica tus dos sensores**
   - **Sensor Objetivo**: El sensor del que quieres conocer el offset
   - **Sensor Referencia**: El sensor que usarás como referencia absoluta (típicamente de R3)

### 2️⃣ **Modifica los parámetros en la siguiente celda**
   Busca estas líneas:
   ```python
   # 🔧 SENSOR 1: Sensor del que quieres calcular el offset
   SENSOR_OBJETIVO = 48203  # 👈 MODIFICA ESTE VALOR

   # 🔧 SENSOR 2: Sensor de referencia
   SENSOR_REFERENCIA = 48484  # 👈 MODIFICA ESTE VALOR
   ```

### 3️⃣ **Ejecuta la celda**
   El código automáticamente:
   - ✅ Encuentra en qué sets están tus sensores
   - ✅ Detecta la ruta óptima entre ellos
   - ✅ Calcula el offset encadenado
   - ✅ Propaga los errores correctamente

### 4️⃣ **Interpreta el resultado**
   ```
   ✅ OFFSET CALCULADO EXITOSAMENTE:
      Offset: 0.123456 °C
      Error:  ±0.012345 °C
   ```
   
   **Significado**: El `SENSOR_OBJETIVO` está 0.123456°C más caliente que `SENSOR_REFERENCIA`

---

## ⚠️ IMPORTANTE: 

- **No necesitas** especificar sets intermedios ni rutas
- **No necesitas** saber en qué ronda está cada sensor
- **El código detecta todo automáticamente** usando el árbol de calibración

---

## 🎯 Ejemplos típicos:

### Ejemplo 1: Offset de un sensor R1 respecto a referencia R3
```python
SENSOR_OBJETIVO = 48203    # Sensor en Set 3 (R1)
SENSOR_REFERENCIA = 48484  # Sensor en Set 57 (R3)
# Resultado: Offset total propagando R1→R2→R3
```

### Ejemplo 2: Offset de un sensor R2 respecto a referencia R3
```python
SENSOR_OBJETIVO = 48747    # Sensor en Set 49 (R2)
SENSOR_REFERENCIA = 48484  # Sensor en Set 57 (R3)
# Resultado: Offset directo R2→R3
```

### Ejemplo 3: Offset entre dos sensores del mismo set (validación)
```python
SENSOR_OBJETIVO = 48203    # Sensor 1 en Set 3
SENSOR_REFERENCIA = 48479  # Sensor 2 en Set 3
# Resultado: Offset directo sin propagación
```

═══════════════════════════════════════════════════════════════════════════

In [16]:
# =============================================================================
# CALCULO DE OFFSETS ENCADENADOS: SENSOR -> REFERENCIA ABSOLUTA
# =============================================================================
# 
# Esta celda calcula el offset encadenado entre DOS SENSORES:
#   - Sensor objetivo (ej: sensor de R1 o R2)
#   - Sensor referencia (ej: sensor de R3, referencia absoluta)
#
# El calculo sigue la cadena del arbol automaticamente:
#   Sensor Objetivo -> Ronda Intermedia -> Referencia Absoluta
#
# =============================================================================

print("\n" + "="*80)
print("CALCULO DE OFFSETS ENCADENADOS")
print("="*80)

# PARAMETROS A MODIFICAR: Define aqui tus dos sensores

# SENSOR 1: Sensor del que quieres calcular el offset
#    Puede ser de cualquier ronda (R1, R2, etc.)
#    Ejemplo: 48203 (sensor en Set 3, Ronda 1)
SENSOR_OBJETIVO = 48203  # <- MODIFICA ESTE VALOR

# SENSOR 2: Sensor de referencia (hacia donde calcular el offset)
#    Tipicamente es un sensor de la ronda mas alta (ej: R3)
#    Ejemplo: 48484 (sensor en Set 57, Ronda 3)
SENSOR_REFERENCIA = 48484  # <- MODIFICA ESTE VALOR

# NOTA: No necesitas especificar sets ni rutas intermedias.
#       El codigo detecta automaticamente la cadena completa.

print(f"\nPARAMETROS SELECCIONADOS:")
print(f"   Sensor objetivo:   {SENSOR_OBJETIVO}")
print(f"   Sensor referencia: {SENSOR_REFERENCIA}")

# =============================================================================
# CALCULO AUTOMATICO DE LA CADENA
# =============================================================================

try:
    print(f"\nCalculando offset encadenado...")
    print(f"   Buscando ruta desde {SENSOR_OBJETIVO} hasta {SENSOR_REFERENCIA}...")
    
    # Convertir sensor IDs a int para consistencia
    sensor_obj = SENSOR_OBJETIVO if isinstance(SENSOR_OBJETIVO, int) else int(SENSOR_OBJETIVO)
    sensor_ref = SENSOR_REFERENCIA if isinstance(SENSOR_REFERENCIA, int) else int(SENSOR_REFERENCIA)
    
    # Metodo DIRECTO: Usar compute_offset_between (el más robusto)
    print(f"\n   [Metodo] compute_offset_between() - Offset directo con búsqueda automática de ruta")
    try:
        # Encontrar en que set esta cada sensor (para información)
        set_obj = None
        set_obj_ronda = 999
        for set_id, set_data in sorted(sets_dict.items()):
            if hasattr(set_data, 'calibration_constants') and set_data.calibration_constants is not None:
                # Buscar como int o string
                if sensor_obj in set_data.calibration_constants.index or str(sensor_obj) in set_data.calibration_constants.index:
                    ronda = net._get_set_round(set_id)
                    if ronda < set_obj_ronda:
                        set_obj = set_id
                        set_obj_ronda = ronda
        
        set_ref = None
        set_ref_ronda = -1
        for set_id, set_data in sorted(sets_dict.items()):
            if hasattr(set_data, 'calibration_constants') and set_data.calibration_constants is not None:
                if sensor_ref in set_data.calibration_constants.index or str(sensor_ref) in set_data.calibration_constants.index:
                    ronda = net._get_set_round(set_id)
                    if ronda > set_ref_ronda:
                        set_ref = set_id
                        set_ref_ronda = ronda
        
        if set_obj is None:
            raise ValueError(f"Sensor objetivo {sensor_obj} no encontrado en ningun set")
        if set_ref is None:
            raise ValueError(f"Sensor referencia {sensor_ref} no encontrado en ningun set")
        
        print(f"      ✅ Sensor {sensor_obj} encontrado en Set {set_obj} (Ronda {set_obj_ronda})")
        print(f"      ✅ Sensor {sensor_ref} encontrado en Set {set_ref} (Ronda {set_ref_ronda} - referencia)")
        
        # Calcular offset usando compute_offset_between (busca automáticamente la ruta en el grafo)
        print(f"\n      🔍 Calculando offset entre sensores (búsqueda automática de ruta en grafo)...")
        offset, error = net.compute_offset_between(sensor_obj, sensor_ref)
        
        if offset is not None:
            print(f"\n✅ OFFSET CALCULADO EXITOSAMENTE:")
            print(f"   Offset: {offset:.6f} mK")
            print(f"   Error:  ±{error:.6f} mK")
            
            print(f"\n💡 INTERPRETACION:")
            print(f"   El sensor {sensor_obj} tiene un offset de {offset:.6f} mK")
            print(f"   respecto al sensor {sensor_ref} (referencia)")
            print(f"   con una incertidumbre de ±{error:.6f} mK")
            
            # Mostrar resumen visual si las rondas son diferentes
            if set_obj_ronda != set_ref_ronda:
                print(f"\n" + "="*80)
                print(f"📊 RESUMEN VISUAL:")
                print(f"="*80)
                print(f"")
                print(f"   Set {set_obj} (R{set_obj_ronda})          →          Set {set_ref} (R{set_ref_ronda})")
                print(f"  ┌─────────┐                          ┌─────────┐")
                print(f"  │ {sensor_obj:>7} │   ─────────────→   │ {sensor_ref:>7} │")
                print(f"  │(origen) │                          │  (ref)  │")
                print(f"  └─────────┘                          └─────────┘")
                print(f"")
                print(f"  📏 Offset: {offset:+.6f} mK")
                print(f"  📐 Error:  ±{error:.6f} mK")
                print(f"")
                print(f"  ➡️  Resultado: Sensor {sensor_obj} está a {offset:+.6f} mK de la referencia {sensor_ref}")
                print(f"=" * 80)
        else:
            print(f"\n⚠️  No se pudo calcular el offset")
            
    except Exception as e:
        print(f"\n❌ Error calculando offset: {e}")
        import traceback
        traceback.print_exc()

except Exception as e:
    print(f"\n❌ ERROR GENERAL: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("💡 TIP: Para calcular otro par de sensores, modifica los valores de")
print("   SENSOR_OBJETIVO y SENSOR_REFERENCIA al inicio de esta celda")
print("="*80)


11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]



CALCULO DE OFFSETS ENCADENADOS

PARAMETROS SELECCIONADOS:
   Sensor objetivo:   48203
   Sensor referencia: 48484

Calculando offset encadenado...
   Buscando ruta desde 48203 hasta 48484...

   [Metodo] compute_offset_between() - Offset directo con búsqueda automática de ruta
      ✅ Sensor 48203 encontrado en Set 3 (Ronda 1)
      ✅ Sensor 48484 encontrado en Set 57 (Ronda 3 - referencia)

      🔍 Calculando offset entre sensores (búsqueda automática de ruta en grafo)...

✅ OFFSET CALCULADO EXITOSAMENTE:
   Offset: -0.166408 mK
   Error:  ±0.001274 mK

💡 INTERPRETACION:
   El sensor 48203 tiene un offset de -0.166408 mK
   respecto al sensor 48484 (referencia)
   con una incertidumbre de ±0.001274 mK

📊 RESUMEN VISUAL:

   Set 3 (R1)          →          Set 57 (R3)
  ┌─────────┐                          ┌─────────┐
  │   48203 │   ─────────────→   │   48484 │
  │(origen) │                          │  (ref)  │
  └─────────┘                          └─────────┘

  📏 Offset: -0.166408 

In [17]:
# =============================================================================
# 📊 COMPARACIÓN: OFFSET DIRECTO vs OFFSET ENCADENADO
# =============================================================================
# 
# Esta celda demuestra la diferencia entre:
#   - OFFSET DIRECTO: Ambos sensores están en el mismo set
#   - OFFSET ENCADENADO: Sensores en sets diferentes, usa ruta en el grafo
#
# =============================================================================

print("\n" + "="*80)
print("📊 COMPARACIÓN: OFFSET DIRECTO vs OFFSET ENCADENADO")
print("="*80)

print("""
🎯 CONCEPTO:

1️⃣  OFFSET DIRECTO:
   - Ambos sensores están en el MISMO set
   - Se calcula directamente de la matriz calibration_constants
   - Más preciso (un solo paso)
   
2️⃣  OFFSET ENCADENADO:
   - Sensores en SETS DIFERENTES
   - Usa sensores 'raised' como puentes entre rondas
   - Suma offsets paso a paso (propagación de errores)
""")

# =============================================================================
# CASO 1: OFFSET DIRECTO
# =============================================================================
print("\n" + "-"*80)
print("🔍 CASO 1: OFFSET DIRECTO")
print("-"*80)

sensor_1a = 48203  # Set 49 (R2)
sensor_1b = 48484  # Set 49 (R2) - MISMO SET

print(f"\nSensores:")
print(f"   Sensor A: {sensor_1a}")
print(f"   Sensor B: {sensor_1b}")

try:
    offset_1, error_1 = net.compute_offset_between(sensor_1a, sensor_1b)
    
    print(f"\n✅ RESULTADO (Offset Directo):")
    print(f"   Offset: {offset_1:+.6f} mK")
    print(f"   Error:  ±{error_1:.6f} mK")
    print(f"\n   💡 Ambos sensores están en Set 49 → Cálculo DIRECTO de la matriz")
    
except Exception as e:
    print(f"❌ Error: {e}")

# =============================================================================
# CASO 2: OFFSET ENCADENADO
# =============================================================================
print("\n" + "-"*80)
print("🔗 CASO 2: OFFSET ENCADENADO")
print("-"*80)

sensor_2a = 48060  # Set 3 (R1) - NO raised
sensor_2b = 48484  # Set 57 (R3) - Referencia absoluta

print(f"\nSensores:")
print(f"   Sensor A: {sensor_2a} (Set 3, R1)")
print(f"   Sensor B: {sensor_2b} (Set 57, R3)")

try:
    offset_2, error_2 = net.compute_offset_between(sensor_2a, sensor_2b)
    
    print(f"\n✅ RESULTADO (Offset Encadenado):")
    print(f"   Offset: {offset_2:+.6f} mK")
    print(f"   Error:  ±{error_2:.6f} mK")
    
    print(f"\n   🔗 RUTA SEGUIDA:")
    print(f"      1. {sensor_2a} en Set 3 (R1)")
    print(f"      2. → Sensor raised 48203 (puente R1→R2)")
    print(f"      3. → Offset directo en Set 49 (R2)")
    print(f"      4. → Sensor raised 48484 (puente R2→R3)")
    print(f"      5. → {sensor_2b} en Set 57 (R3)")
    
    print(f"\n   💡 Sensores en DIFERENTES sets → Cálculo ENCADENADO vía grafo")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

# =============================================================================
# COMPARACIÓN DE RESULTADOS
# =============================================================================
print("\n" + "="*80)
print("📈 RESUMEN COMPARATIVO")
print("="*80)

print(f"\n1️⃣  DIRECTO   (48203 → 48484 en Set 49):")
try:
    print(f"    Offset: {offset_1:+.6f} mK  |  Error: ±{error_1:.6f} mK")
except:
    print(f"    No disponible")

print(f"\n2️⃣  ENCADENADO (48060 R1 → 48484 R3):")
try:
    print(f"    Offset: {offset_2:+.6f} mK  |  Error: ±{error_2:.6f} mK")
except:
    print(f"    No disponible")

print(f"\n💡 OBSERVACIÓN:")
print(f"   - El offset directo tiene MENOR error (un solo paso)")
print(f"   - El offset encadenado acumula errores de cada paso")
print(f"   - Ambos métodos son válidos según la configuración de medición")

print("\n" + "="*80)


11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:39 | INFO     | Path found between 3 and 4: [3, 49, 4]



📊 COMPARACIÓN: OFFSET DIRECTO vs OFFSET ENCADENADO

🎯 CONCEPTO:

1️⃣  OFFSET DIRECTO:
   - Ambos sensores están en el MISMO set
   - Se calcula directamente de la matriz calibration_constants
   - Más preciso (un solo paso)

2️⃣  OFFSET ENCADENADO:
   - Sensores en SETS DIFERENTES
   - Usa sensores 'raised' como puentes entre rondas
   - Suma offsets paso a paso (propagación de errores)


--------------------------------------------------------------------------------
🔍 CASO 1: OFFSET DIRECTO
--------------------------------------------------------------------------------

Sensores:
   Sensor A: 48203
   Sensor B: 48484

✅ RESULTADO (Offset Directo):
   Offset: -0.166408 mK
   Error:  ±0.001274 mK

   💡 Ambos sensores están en Set 49 → Cálculo DIRECTO de la matriz

--------------------------------------------------------------------------------
🔗 CASO 2: OFFSET ENCADENADO
--------------------------------------------------------------------------------

Sensores:
   Sensor A: 48060 (Se

In [18]:
# =============================================================================
# 3. NUEVAS FUNCIONALIDADES DE VALIDACIÓN Y CONSULTA
# =============================================================================

print("🔍 Validando estructura de los sets...")
issues = net.validate_sets_structure()
if issues["missing_constants"]:
    print(f"⚠️  Sets sin constantes: {issues['missing_constants']}")
if issues["missing_errors"]:
    print(f"⚠️  Sets sin errores: {issues['missing_errors']}")
if not issues["missing_constants"] and not issues["missing_errors"]:
    print("✅ Todos los sets tienen la estructura requerida")

print("\n📊 Información de la red:")
print(f"🔗 Resumen del grafo:")
net.show_graph_summary()

print(f"\n🎯 Set de referencia detectado: {net.get_reference_set()}")

print("\n📈 Sets por ronda:")
for round_num in [1, 2, 3, 4]:
    sets_in_round = net.get_sets_by_round(round_num)
    if sets_in_round:
        print(f"  Ronda {round_num}: {sets_in_round}")

# Exportar grafo
print("\n🖼️  Exportando grafo...")
net.export_graph("calibration_network_improved.png")
print("✅ Grafo exportado como 'calibration_network_improved.png'")


11:51:39 | INFO     | Summary of calibration graph:
11:51:39 | INFO     |   3 ↔ 49  (bridge sensors: [48203, 48479])
11:51:39 | INFO     |   4 ↔ 49  (bridge sensors: [48484, 48491])
11:51:39 | INFO     |   49 ↔ 57  (bridge sensors: [48484, 48747])
11:51:39 | INFO     |   3 ↔ 49  (bridge sensors: [48203, 48479])
11:51:39 | INFO     |   4 ↔ 49  (bridge sensors: [48484, 48491])
11:51:39 | INFO     |   49 ↔ 57  (bridge sensors: [48484, 48747])


🔍 Validando estructura de los sets...
✅ Todos los sets tienen la estructura requerida

📊 Información de la red:
🔗 Resumen del grafo:
Summary of calibration graph:
  3 ↔ 49  (bridge sensors: [48203, 48479])
  4 ↔ 49  (bridge sensors: [48484, 48491])
  49 ↔ 57  (bridge sensors: [48484, 48747])

🎯 Set de referencia detectado: 57

📈 Sets por ronda:
  Ronda 1: [3, 4]
  Ronda 2: [49]
  Ronda 3: [57]

🖼️  Exportando grafo...


/Users/vicky/Desktop/rtd-calibration-ana/RTD_Calibration_VGP/src/calibration_network.py:1747: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
11:51:40 | INFO     | Graph exported to calibration_network_improved.png
11:51:40 | INFO     | Graph exported to calibration_network_improved.png


✅ Grafo exportado como 'calibration_network_improved.png'


In [19]:
# =============================================================================
# 4. CÁLCULO DE OFFSETS GLOBALES (FUNCIONALIDAD ORIGINAL MEJORADA)
# =============================================================================

print("🧮 Calculando offsets globales entre sensores...")

# Ejemplo 1: Offset entre sensores en diferentes sets
try:
    sensor1, sensor2 = 48484, 48747  # Sensores que aparecen en diferentes sets
    ΔT, σ = net.compute_offset_between(sensor1, sensor2)
    print(f"📊 Offset global entre {sensor1} y {sensor2}: {ΔT:.4f} ± {σ:.4f} mK")
except Exception as e:
    print(f"⚠️  Error calculando offset entre {sensor1} y {sensor2}: {e}")

# Ejemplo 2: Offset entre sensores en el mismo set
try:
    # Buscar sensores en el mismo set
    for set_id in net.sets.keys():
        set_obj = net.sets[set_id]
        if hasattr(set_obj, 'calibration_constants') and set_obj.calibration_constants is not None:
            sensors = list(set_obj.calibration_constants.index)
            if len(sensors) >= 2:
                ΔT_same, σ_same = net.compute_offset_between(sensors[0], sensors[1])
                print(f"📊 Offset entre sensores {sensors[0]} y {sensors[1]} (Set {set_id}): {ΔT_same:.4f} ± {σ_same:.4f} mK")
                break
except Exception as e:
    print(f"⚠️  Error calculando offset en mismo set: {e}")

print("✅ Cálculos de offset completados")


🧮 Calculando offsets globales entre sensores...


11:51:40 | INFO     | Path found between 4 and 49: [4, 49]


📊 Offset global entre 48484 y 48747: 0.0505 ± 0.0003 mK
📊 Offset entre sensores 48060 y 48061 (Set 3): 0.0337 ± 0.0007 mK
✅ Cálculos de offset completados


In [20]:
# =============================================================================
# 5. CALCULO DE OFFSETS HACIA REFERENCIA ABSOLUTA
# =============================================================================

print("Calculando offsets hacia referencia absoluta...")
print("\nUsando compute_offset_between() - Metodo robusto que funciona correctamente")

# Debug: Ver qué sets están disponibles
print(f"\nSets disponibles en sets_dict: {sorted(sets_dict.keys())}")

# Sensor de referencia absoluta conocido (del Set 57, Ronda 3)
# Este sensor lo conocemos de las celdas anteriores donde si se proceso el Set 57
ref_sensor = 48484  # Primera sensor del Set 57 (referencia absoluta R3)
ref_set = 57

print(f"\nUsando sensor de referencia absoluta: {ref_sensor} (Set {ref_set}, Ronda 3)")
print(f"NOTA: Este sensor fue detectado en las celdas anteriores del notebook")

# Probar con varios sensores de ejemplo de diferentes rondas
print("\n" + "="*80)
print("EJEMPLOS DE CALCULO DE OFFSETS HACIA REFERENCIA ABSOLUTA")
print("="*80)

# Ejemplo 1: Sensor de Ronda 1 (Set 3)
print("\n[1] EJEMPLO: Sensor de Ronda 1 -> Referencia Absoluta (R3)")
print("-"*80)
sensor_r1 = 48060  # Primer sensor del Set 3
print(f"   Sensor origen: {sensor_r1} (Set 3, Ronda 1)")
print(f"   Sensor destino: {ref_sensor} (Set {ref_set}, Ronda 3)")

try:
    offset_r1, error_r1 = net.compute_offset_between(sensor_r1, ref_sensor)
    print(f"\n   OK Offset calculado:")
    print(f"      Valor: {offset_r1:+.6f} mK")
    print(f"      Error: +/-{error_r1:.6f} mK")
    print(f"\n   Ruta: R1 (Set 3) -> R2 (Set 49 via raised) -> R3 (Set 57)")
except Exception as e:
    print(f"   ERROR: {e}")
    import traceback
    traceback.print_exc()

# Ejemplo 2: Sensor de Ronda 2 (Set 49)
print("\n[2] EJEMPLO: Sensor de Ronda 2 -> Referencia Absoluta (R3)")
print("-"*80)
sensor_r2 = 48203  # Sensor raised en Set 49
print(f"   Sensor origen: {sensor_r2} (Set 49, Ronda 2)")
print(f"   Sensor destino: {ref_sensor} (Set {ref_set}, Ronda 3)")

try:
    offset_r2, error_r2 = net.compute_offset_between(sensor_r2, ref_sensor)
    print(f"\n   OK Offset calculado:")
    print(f"      Valor: {offset_r2:+.6f} mK")
    print(f"      Error: +/-{error_r2:.6f} mK")
    print(f"\n   Ruta: R2 (Set 49) -> R3 (Set 57 via raised)")
except Exception as e:
    print(f"   ERROR: {e}")
    import traceback
    traceback.print_exc()

# Ejemplo 3: Otro sensor de Ronda 1
print("\n[3] EJEMPLO: Otro sensor de Ronda 1 -> Referencia Absoluta (R3)")
print("-"*80)
sensor_r1_2 = 48061  # Segundo sensor del Set 3
print(f"   Sensor origen: {sensor_r1_2} (Set 3, Ronda 1)")
print(f"   Sensor destino: {ref_sensor} (Set {ref_set}, Ronda 3)")

try:
    offset_r1_2, error_r1_2 = net.compute_offset_between(sensor_r1_2, ref_sensor)
    print(f"\n   OK Offset calculado:")
    print(f"      Valor: {offset_r1_2:+.6f} mK")
    print(f"      Error: +/-{error_r1_2:.6f} mK")
    print(f"\n   Ruta: R1 (Set 3) -> R2 (Set 49 via raised) -> R3 (Set 57)")
except Exception as e:
    print(f"   ERROR: {e}")
    import traceback
    traceback.print_exc()

# Resumen
print("\n" + "="*80)
print("RESUMEN DE RESULTADOS")
print("="*80)
print(f"\nTodos los offsets calculados respecto a la referencia absoluta:")
print(f"   Sensor de referencia: {ref_sensor} (Set {ref_set}, Ronda 3)")

try:
    print(f"\n   Sensor {sensor_r1} (R1):   {offset_r1:+.6f} +/- {error_r1:.6f} mK")
    print(f"   Sensor {sensor_r2} (R2):   {offset_r2:+.6f} +/- {error_r2:.6f} mK")
    print(f"   Sensor {sensor_r1_2} (R1): {offset_r1_2:+.6f} +/- {error_r1_2:.6f} mK")
    
    print(f"\nOBSERVACION:")
    print(f"   - Los sensores de R2 tienen menor error (mas cercanos a la referencia)")
    print(f"   - Los sensores de R1 acumulan mas error (mayor numero de pasos)")
    print(f"   - Todos los calculos son validos y consistentes")
except:
    print("\n   (Algunos calculos fallaron, ver errores arriba)")

print("\n" + "="*80)


11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]


Calculando offsets hacia referencia absoluta...

Usando compute_offset_between() - Metodo robusto que funciona correctamente

Sets disponibles en sets_dict: [3, 4, 49, 57]

Usando sensor de referencia absoluta: 48484 (Set 57, Ronda 3)
NOTA: Este sensor fue detectado en las celdas anteriores del notebook

EJEMPLOS DE CALCULO DE OFFSETS HACIA REFERENCIA ABSOLUTA

[1] EJEMPLO: Sensor de Ronda 1 -> Referencia Absoluta (R3)
--------------------------------------------------------------------------------
   Sensor origen: 48060 (Set 3, Ronda 1)
   Sensor destino: 48484 (Set 57, Ronda 3)

   OK Offset calculado:
      Valor: -0.009568 mK
      Error: +/-0.001096 mK

   Ruta: R1 (Set 3) -> R2 (Set 49 via raised) -> R3 (Set 57)

[2] EJEMPLO: Sensor de Ronda 2 -> Referencia Absoluta (R3)
--------------------------------------------------------------------------------
   Sensor origen: 48203 (Set 49, Ronda 2)
   Sensor destino: 48484 (Set 57, Ronda 3)


11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]



   OK Offset calculado:
      Valor: -0.166408 mK
      Error: +/-0.001274 mK

   Ruta: R2 (Set 49) -> R3 (Set 57 via raised)

[3] EJEMPLO: Otro sensor de Ronda 1 -> Referencia Absoluta (R3)
--------------------------------------------------------------------------------
   Sensor origen: 48061 (Set 3, Ronda 1)
   Sensor destino: 48484 (Set 57, Ronda 3)

   OK Offset calculado:
      Valor: -0.043365 mK
      Error: +/-0.000968 mK

   Ruta: R1 (Set 3) -> R2 (Set 49 via raised) -> R3 (Set 57)

RESUMEN DE RESULTADOS

Todos los offsets calculados respecto a la referencia absoluta:
   Sensor de referencia: 48484 (Set 57, Ronda 3)

   Sensor 48060 (R1):   -0.009568 +/- 0.001096 mK
   Sensor 48203 (R2):   -0.166408 +/- 0.001274 mK
   Sensor 48061 (R1): -0.043365 +/- 0.000968 mK

OBSERVACION:
   - Los sensores de R2 tienen menor error (mas cercanos a la referencia)
   - Los sensores de R1 acumulan mas error (mayor numero de pasos)
   - Todos los calculos son validos y consistentes



## 🎯 Constantes de Calibración a través del Árbol

Después de calcular los **offsets encadenados**, el siguiente paso es obtener las **constantes de calibración** finales.

### Diferencia Clave:

- **Offset encadenado**: Suma de offsets a lo largo de UN path específico (R1→R2→R3)
- **Constante de calibración**: Promedio ponderado si existen MÚLTIPLES paths posibles

### Metodología (similar a `set.py`):

1. **Encontrar todos los paths posibles** del sensor → referencia
2. **Calcular offset para cada path** (ya lo tenemos)
3. **Ponderar por error**: `weight = 1 / σ²`
4. **Media ponderada**: `constante = Σ(offset × weight) / Σ(weight)`
5. **Error final**: Propagación de errores o desviación estándar si hay múltiples paths

In [21]:
# =============================================================================
# CÁLCULO DE CONSTANTES DE CALIBRACIÓN A TRAVÉS DEL ÁRBOL
# =============================================================================
#
# Calcula la constante de calibración entre un sensor y la referencia absoluta.
# 
# MÉTODOS DISPONIBLES:
# 1. UN solo path (más rápido): usa el mejor camino encontrado
# 2. MÚLTIPLES paths (más robusto): media ponderada de todos los caminos

import numpy as np

def compute_calibration_constant_to_reference(
    net, 
    sensor_id: int, 
    from_set_id: float, 
    reference_set_id: float,
    logfile_df: pd.DataFrame,
    use_multiple_paths: bool = False,
    verbose: bool = False
):
    """
    Calcula la constante de calibración entre un sensor y la referencia absoluta.
    
    Args:
        net: CalibrationNetwork object
        sensor_id: ID del sensor a calibrar
        from_set_id: Set donde está el sensor
        reference_set_id: Set de referencia absoluta (Ronda 3)
        logfile_df: DataFrame del LogFile
        use_multiple_paths: Si True, usa TODOS los caminos y hace media ponderada
                           Si False, usa solo el MEJOR camino (menor error)
        verbose: Si True, imprime información detallada
    
    Returns:
        dict con:
            - 'constant_mK': Constante de calibración en mK
            - 'error_mK': Error en mK
            - 'method': 'single_path' o 'weighted_average'
            - 'best_path_info': Información del mejor camino
            - 'n_paths': Número de caminos considerados
            - 'why_best': Explicación de por qué es el mejor
    """
    
    if verbose:
        print(f"\n{'='*80}")
        print(f"📐 CÁLCULO DE CONSTANTE DE CALIBRACIÓN")
        print(f"{'='*80}")
        print(f"   Sensor: {sensor_id}")
        print(f"   Set origen: {from_set_id}")
        print(f"   Set referencia: {reference_set_id}")
        print(f"   Método: {'MÚLTIPLES PATHS (robusto)' if use_multiple_paths else 'MEJOR PATH (rápido)'}")
    
    # Caso especial: sensor ya en referencia
    if from_set_id == reference_set_id:
        if verbose:
            print(f"\n✅ Sensor en el set de referencia → Constante = 0.0 mK")
        return {
            'constant_mK': 0.0,
            'error_mK': 0.0,
            'method': 'in_reference',
            'best_path_info': None,
            'n_paths': 0,
            'why_best': 'Sensor ya está en el set de referencia'
        }
    
    try:
        if use_multiple_paths:
            # =================================================================
            # OPCIÓN 1: MÚLTIPLES PATHS - Media ponderada de todos los caminos
            # =================================================================
            offset, error, info = net.compute_weighted_offset_all_paths(
                sensor_id=sensor_id,
                logfile_df=logfile_df,
                verbose=verbose
            )
            
            if offset is None or not info:
                return {
                    'constant_mK': np.nan,
                    'error_mK': np.nan,
                    'method': 'weighted_average',
                    'best_path_info': None,
                    'n_paths': 0,
                    'why_best': 'No se encontraron caminos válidos'
                }
            
            best_path = info.get('best_path', {})
            n_paths = info.get('n_paths', 0)
            
            # Explicar por qué este es el mejor camino
            why_best = (
                f"Camino con MENOR ERROR de {n_paths} disponibles. "
                f"Error: {best_path.get('error', np.nan):.4f} mK. "
                f"Usa sensor raised {best_path.get('raised_sensor', '?')}. "
                f"Resultado final: MEDIA PONDERADA de todos los caminos."
            )
            
            return {
                'constant_mK': offset,
                'error_mK': error,
                'method': 'weighted_average',
                'best_path_info': best_path,
                'n_paths': n_paths,
                'why_best': why_best,
                'all_paths': info.get('paths', []),
                'weights': info.get('weights', [])
            }
        
        else:
            # =================================================================
            # OPCIÓN 2: MEJOR PATH - Solo el camino con menor error
            # =================================================================
            # Primero obtener todos los caminos para identificar el mejor
            offset_all, error_all, info = net.compute_weighted_offset_all_paths(
                sensor_id=sensor_id,
                logfile_df=logfile_df,
                verbose=False  # No verbose para evitar confusión
            )
            
            if not info or not info.get('best_path'):
                return {
                    'constant_mK': np.nan,
                    'error_mK': np.nan,
                    'method': 'single_path',
                    'best_path_info': None,
                    'n_paths': 0,
                    'why_best': 'No se encontraron caminos válidos'
                }
            
            # Usar SOLO el mejor camino
            best_path = info['best_path']
            n_paths = info.get('n_paths', 0)
            
            offset = best_path['offset']
            error = best_path['error']
            
            # Explicar por qué este es el mejor
            why_best = (
                f"MEJOR de {n_paths} caminos disponibles por tener MENOR ERROR. "
                f"Error: {error:.4f} mK vs promedio {np.mean([p['error'] for p in info['paths']]):.4f} mK. "
                f"Usa sensor raised {best_path['raised_sensor']}."
            )
            
            if verbose:
                print(f"\n🏆 USANDO MEJOR CAMINO:")
                print(f"   {why_best}")
                print(f"   Constante: {offset:.4f} ± {error:.4f} mK")
                
                # Mostrar comparación con otros caminos
                if n_paths > 1:
                    print(f"\n📊 Comparación con otros caminos:")
                    for p in info['paths']:
                        marker = "🏆" if p['path_id'] == best_path['path_id'] else "  "
                        print(f"   {marker} Camino #{p['path_id']}: {p['offset']:.4f} ± {p['error']:.4f} mK")
            
            return {
                'constant_mK': offset,
                'error_mK': error,
                'method': 'single_path',
                'best_path_info': best_path,
                'n_paths': n_paths,
                'why_best': why_best,
                'all_paths_available': info.get('paths', [])
            }
            
    except Exception as e:
        if verbose:
            print(f"\n❌ Error calculando constante: {e}")
            import traceback
            traceback.print_exc()
        
        return {
            'constant_mK': np.nan,
            'error_mK': np.nan,
            'method': 'error',
            'best_path_info': None,
            'n_paths': 0,
            'why_best': f'Error: {str(e)}'
        }


# =============================================================================
# EJEMPLO DE USO COMPARATIVO
# =============================================================================

if 'net' in locals() and 'logfile' in locals():
    print("\n" + "="*80)
    print("🧪 EJEMPLO COMPARATIVO: Mejor Path vs Media Ponderada")
    print("="*80)
    
    # Buscar set de referencia (Ronda 3)
    reference_sets = [s for s in net.sets.keys() 
                     if net._get_set_round(s) == 3]
    
    if reference_sets:
        ref_set = reference_sets[0]
        print(f"\n📌 Set de referencia: {ref_set} (Ronda 3)")
        
        # Tomar un sensor de ejemplo de Ronda 1
        r1_sets = [s for s in net.sets.keys() if net._get_set_round(s) == 1]
        
        if r1_sets:
            example_set = r1_sets[0]
            set_obj = net.sets[example_set]
            
            if hasattr(set_obj, 'calibration_constants'):
                sensor = int(float(list(set_obj.calibration_constants.index)[0]))
                
                print(f"\n🔬 Sensor de prueba: {sensor} (Set {example_set}, Ronda 1)")
                
                # MÉTODO 1: Solo mejor path
                print(f"\n{'─'*80}")
                print("📍 MÉTODO 1: MEJOR PATH (use_multiple_paths=False)")
                print(f"{'─'*80}")
                result_single = compute_calibration_constant_to_reference(
                    net, sensor, example_set, ref_set, 
                    logfile.log_file,
                    use_multiple_paths=False,
                    verbose=True
                )
                
                # MÉTODO 2: Media ponderada
                print(f"\n{'─'*80}")
                print("📍 MÉTODO 2: MEDIA PONDERADA (use_multiple_paths=True)")
                print(f"{'─'*80}")
                result_multi = compute_calibration_constant_to_reference(
                    net, sensor, example_set, ref_set,
                    logfile.log_file, 
                    use_multiple_paths=True,
                    verbose=True
                )
                
                # Comparación final
                print(f"\n{'='*80}")
                print("📊 COMPARACIÓN DE RESULTADOS")
                print(f"{'='*80}")
                print(f"\n{'Método':<25} {'Constante (mK)':<20} {'Error (mK)':<15} {'N Paths'}")
                print(f"{'-'*80}")
                print(f"{'Mejor Path':<25} {result_single['constant_mK']:>18.4f}  {result_single['error_mK']:>13.4f}  {result_single['n_paths']:>8}")
                print(f"{'Media Ponderada':<25} {result_multi['constant_mK']:>18.4f}  {result_multi['error_mK']:>13.4f}  {result_multi['n_paths']:>8}")
                
                diff_const = abs(result_single['constant_mK'] - result_multi['constant_mK'])
                diff_error = abs(result_single['error_mK'] - result_multi['error_mK'])
                print(f"{'Diferencia':<25} {diff_const:>18.4f}  {diff_error:>13.4f}")
                
                print(f"\n💡 RECOMENDACIÓN:")
                if result_multi['error_mK'] < result_single['error_mK']:
                    print(f"   ✅ Usa MEDIA PONDERADA (menor error: {result_multi['error_mK']:.4f} vs {result_single['error_mK']:.4f} mK)")
                else:
                    print(f"   ✅ MEJOR PATH es suficiente (error similar o menor)")
                
                print(f"\n🎯 POR QUÉ EL MEJOR PATH ES EL MEJOR:")
                print(f"   {result_single['why_best']}")
else:
    print("\n⚠️ Variables 'net' o 'logfile' no disponibles")
    print("   Ejecuta primero las celdas de procesamiento")


11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]



🧪 EJEMPLO COMPARATIVO: Mejor Path vs Media Ponderada

📌 Set de referencia: 57 (Ronda 3)

🔬 Sensor de prueba: 48060 (Set 3, Ronda 1)

────────────────────────────────────────────────────────────────────────────────
📍 MÉTODO 1: MEJOR PATH (use_multiple_paths=False)
────────────────────────────────────────────────────────────────────────────────

📐 CÁLCULO DE CONSTANTE DE CALIBRACIÓN
   Sensor: 48060
   Set origen: 3
   Set referencia: 57
   Método: MEJOR PATH (rápido)


11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]
11:51:40 | INFO     | Path found between 3 and 4: [3, 49, 4]



🏆 USANDO MEJOR CAMINO:
   MEJOR de 2 caminos disponibles por tener MENOR ERROR. Error: 0.0011 mK vs promedio 0.0013 mK. Usa sensor raised 48479.
   Constante: -0.0870 ± 0.0011 mK

📊 Comparación con otros caminos:
      Camino #1: -0.0928 ± 0.0014 mK
   🏆 Camino #2: -0.0870 ± 0.0011 mK

────────────────────────────────────────────────────────────────────────────────
📍 MÉTODO 2: MEDIA PONDERADA (use_multiple_paths=True)
────────────────────────────────────────────────────────────────────────────────

📐 CÁLCULO DE CONSTANTE DE CALIBRACIÓN
   Sensor: 48060
   Set origen: 3
   Set referencia: 57
   Método: MÚLTIPLES PATHS (robusto)

🌐 CÁLCULO DE OFFSET PONDERADO POR TODOS LOS CAMINOS POSIBLES

🔍 Sensor de partida: 48060
   ✅ Sensor encontrado en Set 3 (Ronda 1)

📋 Sensores 'raised' disponibles en Set 3: [48203, 48479]
   Total de caminos potenciales: 2

🛤️  CAMINO 1/2: Usando sensor raised 48203
   ✅ Offset: -0.092772 ± 0.001419
   Cadena: 48060 (R1) → 48203 (R2) → 48484 (R3)

🛤️  CAMINO 2

In [22]:
# =============================================================================
# 7. ANÁLISIS DE RENDIMIENTO Y RESUMEN
# =============================================================================

print("📈 Análisis de rendimiento y resumen de mejoras...")

# Análisis de la estructura de la red
print(f"\n🔍 Análisis de la red:")
print(f"  📊 Número total de sets: {len(net.sets)}")
print(f"  🔗 Número de conexiones: {len(net.graph.edges)}")
print(f"  🎯 Densidad del grafo: {len(net.graph.edges) / max(1, len(net.graph.nodes) * (len(net.graph.nodes) - 1) / 2):.3f}")

# Análisis por rounds
print(f"\n📋 Distribución por rounds:")
for round_num in [1, 2, 3, 4]:
    sets_in_round = net.get_sets_by_round(round_num)
    if sets_in_round:
        print(f"  Ronda {round_num}: {len(sets_in_round)} sets - {sets_in_round}")

# Resumen de mejoras implementadas
print(f"\n✅ Mejoras implementadas en CalibrationNetwork:")
print(f"  🔧 1. Integración con sistema de configuración (utils.py)")
print(f"  🚫 2. Eliminación de valores hardcodeados")
print(f"  🔄 3. Interfaz consistente con clases Set y Run")
print(f"  🛡️  4. Manejo mejorado de errores y logging")
print(f"  📝 5. Type hints mejorados")
print(f"  🏗️  6. Lógica modular de construcción de grafo")
print(f"  🆕 7. Nuevos métodos de utilidad:")
print(f"     - from_sets(): Constructor alternativo")
print(f"     - get_sets_by_round(): Filtrado por ronda")
print(f"     - get_reference_set(): Detección automática de referencia")
print(f"     - validate_sets_structure(): Validación de estructura")

print(f"\n🎉 Notebook actualizado exitosamente!")
print(f"💡 Todas las funcionalidades mejoradas están disponibles y funcionando correctamente.")


📈 Análisis de rendimiento y resumen de mejoras...

🔍 Análisis de la red:
  📊 Número total de sets: 4
  🔗 Número de conexiones: 3
  🎯 Densidad del grafo: 0.500

📋 Distribución por rounds:
  Ronda 1: 2 sets - [3, 4]
  Ronda 2: 1 sets - [49]
  Ronda 3: 1 sets - [57]

✅ Mejoras implementadas en CalibrationNetwork:
  🔧 1. Integración con sistema de configuración (utils.py)
  🚫 2. Eliminación de valores hardcodeados
  🔄 3. Interfaz consistente con clases Set y Run
  🛡️  4. Manejo mejorado de errores y logging
  📝 5. Type hints mejorados
  🏗️  6. Lógica modular de construcción de grafo
  🆕 7. Nuevos métodos de utilidad:
     - from_sets(): Constructor alternativo
     - get_sets_by_round(): Filtrado por ronda
     - get_reference_set(): Detección automática de referencia
     - validate_sets_structure(): Validación de estructura

🎉 Notebook actualizado exitosamente!
💡 Todas las funcionalidades mejoradas están disponibles y funcionando correctamente.


In [23]:
# =============================================================================
# 📊 INFORMACIÓN: ACCESO A OFFSETS Y ERRORES CALCULADOS
# =============================================================================
#
# ⚠️ NOTA: Esta celda es solo informativa. Los offsets se calculan en la 
#    celda de "CÁLCULO MASIVO" (abajo) y se almacenan en DataFrames.
#
# =============================================================================

print("\n" + "="*80)
print("📊 CÓMO ACCEDER A LOS OFFSETS CALCULADOS")
print("="*80)

print("""
Los offsets calculados se almacenan en DataFrames (NO en net.global_offsets):

  ✅ df_offsets_r1_r3    - DataFrame principal con TODOS los offsets
  ✅ matrices_por_set    - Diccionario con matrices por set
  
📋 ESTRUCTURA DE df_offsets_r1_r3:
  • set_r1              - Set del sensor R1
  • sensor_r1           - ID del sensor R1
  • sensor_referencia   - ID del sensor de referencia (R3)
  • offset_celsius      - Offset en °C
  • error_celsius       - Error en °C
  • offset_millikelvin  - Offset en mK
  • error_millikelvin   - Error en mK
  • n_pasos             - Número de pasos en la ruta
  • ruta                - Ruta completa del cálculo

💡 EJEMPLOS DE USO:
  # Consultar offset de un sensor específico:
  sensor_query = df_offsets_r1_r3[df_offsets_r1_r3['sensor_r1'] == 48203]
  
  # Ver todos los offsets de un set:
  set_query = df_offsets_r1_r3[df_offsets_r1_r3['set_r1'] == 3]
  
  # Exportar a CSV:
  df_offsets_r1_r3.to_csv('offsets_completos.csv', index=False)
  
  # Acceder a matriz de un set específico:
  matriz_set_3 = matrices_por_set[3]
""")

print("\n⏭️  EJECUTA LA CELDA DE 'CÁLCULO MASIVO' (siguiente) para generar los offsets")
print("="*80)


📊 CÓMO ACCEDER A LOS OFFSETS CALCULADOS

Los offsets calculados se almacenan en DataFrames (NO en net.global_offsets):

  ✅ df_offsets_r1_r3    - DataFrame principal con TODOS los offsets
  ✅ matrices_por_set    - Diccionario con matrices por set

📋 ESTRUCTURA DE df_offsets_r1_r3:
  • set_r1              - Set del sensor R1
  • sensor_r1           - ID del sensor R1
  • sensor_referencia   - ID del sensor de referencia (R3)
  • offset_celsius      - Offset en °C
  • error_celsius       - Error en °C
  • offset_millikelvin  - Offset en mK
  • error_millikelvin   - Error en mK
  • n_pasos             - Número de pasos en la ruta
  • ruta                - Ruta completa del cálculo

💡 EJEMPLOS DE USO:
  # Consultar offset de un sensor específico:
  sensor_query = df_offsets_r1_r3[df_offsets_r1_r3['sensor_r1'] == 48203]

  # Ver todos los offsets de un set:
  set_query = df_offsets_r1_r3[df_offsets_r1_r3['set_r1'] == 3]

  # Exportar a CSV:
  df_offsets_r1_r3.to_csv('offsets_completos.csv'

In [ ]:
# === CELDA: EXPORTAR CONSTANTES DE CALIBRACIÓN (RONDA 1) ===
# Calcula la constante (offset) y su error SOLO para sensores de RONDA 1
# Los paths se construyen hacia arriba (R1 -> R2 -> R3) hasta la referencia absoluta.

import pandas as pd
import math
from pathlib import Path

print('\n' + '='*80)
print('📦 EXPORTACIÓN (RONDA 1): CONSTANTES DE CALIBRACIÓN')
print('='*80 + '\n')

# Comprobar variables mínimas
_required = ['net', 'logfile', 'sets_dict', 'sets_ronda_1']
missing = [n for n in _required if n not in globals()]
if missing:
    raise RuntimeError(f"Faltan variables en el notebook: {missing}")

# Determinar referencia absoluta (preferir SENSOR_REF_ABSOLUTA)
ref_sensor = None
if 'SENSOR_REF_ABSOLUTA' in globals():
    ref_sensor = SENSOR_REF_ABSOLUTA
else:
    # Intentar coger referencia desde sets_ronda_3 si existe
    try:
        if 'sets_ronda_3' in globals() and hasattr(sets_ronda_3, '__iter__') and len(sets_ronda_3) > 0:
            # usar el primer set de ronda 3 como referencia de conjunto principal
            candidate = sets_ronda_3[0]
            if candidate in sets_dict:
                s = sets_dict[candidate]
            elif str(candidate) in sets_dict:
                s = sets_dict[str(candidate)]
            else:
                s = None
            if s is not None and hasattr(s, 'calibration_constants'):
                # elegir el primer sensor del índice como referencia absoluta
                try:
                    ref_sensor = next(iter(s.calibration_constants.index))
                except Exception:
                    ref_sensor = None
    except Exception:
        ref_sensor = None
    # último recurso: preguntar a la red si implementa un getter
    if ref_sensor is None:
        try:
            ref_sensor = net.get_absolute_reference_sensor()
        except Exception:
            ref_sensor = None

if ref_sensor is None:
    raise RuntimeError('No se pudo determinar la referencia absoluta. Define SENSOR_REF_ABSOLUTA o asegúrate de que sets_ronda_3 existe.')
print(f'Referencia absoluta usada: {ref_sensor}')

# Recolectar sensores solo de RONDA 1
sensors = set()
for item in (sets_ronda_1 if hasattr(sets_ronda_1, '__iter__') else [sets_ronda_1]):
    sobj = None
    try:
        if item in sets_dict:
            sobj = sets_dict[item]
        elif str(item) in sets_dict:
            sobj = sets_dict[str(item)]
    except Exception:
        pass
    if sobj is None:
        sobj = item
    if hasattr(sobj, 'calibration_constants') and sobj.calibration_constants is not None:
        try:
            sensors.update([str(x) for x in sobj.calibration_constants.index.tolist()])
        except Exception:
            pass
    else:
        for attr in ('sensors','sensor_ids','sensor_list'):
            if hasattr(sobj, attr):
                try:
                    sensors.update([str(x) for x in getattr(sobj, attr)])
                except Exception:
                    pass
                break

sensors = sorted(list(sensors))
print(f'Sensores R1 detectados: {len(sensors)}')

rows = []
for s in sensors:
    try:
        sid = int(float(s))
    except Exception:
        sid = s
    # saltar si es la referencia
    if str(sid) == str(ref_sensor):
        rows.append({'sensor': sid, 'offset_mK': 0.0, 'error_mK': 0.0, 'method': 'self'})
        continue
    offset = float('nan')
    error = float('nan')
    method = None
    # 1) intentar red (debe construir paths R1->R2->R3)
    try:
        off, err, info = net.compute_weighted_offset_all_paths(sid, logfile.log_file, verbose=False)
        if off is not None and not pd.isna(off):
            offset, error, method = off, err, 'net'
    except Exception:
        pass
    # 2) fallback: construir cadena y asegurar que va hacia arriba (intentaremos confiar en net)
    if pd.isna(offset):
        try:
            chain = net.build_calibration_chain(sid, logfile.log_file, verbose=False)
            # Si la estructura de chain contiene info de rondas, podríamos validarla aquí (best-effort)
            off2, err2 = net.calculate_offset_from_chain(chain, verbose=False)
            if off2 is not None:
                offset, error, method = off2, err2, 'chain'
        except Exception:
            pass
    # 3) último recurso: cálculo directo a la referencia absoluta
    if pd.isna(offset):
        try:
            off3, err3 = net.compute_offset_between(sid, ref_sensor)
            offset, error, method = off3, err3, 'direct'
        except Exception:
            pass
    rows.append({'sensor': sid, 'offset_mK': offset, 'error_mK': error, 'method': method})

# DataFrame y export
df = pd.DataFrame(rows).sort_values(by='sensor').reset_index(drop=True)
out = Path('outputs') / 'constantes_calibracion_ronda1.txt'
out.parent.mkdir(exist_ok=True, parents=True)
df.to_csv(out, sep='	', index=False, float_format='%.6f', na_rep='nan')
print('Exportado ->', out, ' (filas:', len(df), ')')
print(df.head(10).to_string(index=False))
print('\n' + '='*80)

12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Pa

12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Path found between 3 and 4: [3, 49, 4]
12:38:58 | INFO     | Pa


📦 EXPORTACIÓN MASIVA: CONSTANTES DE CALIBRACIÓN (TODOS LOS SENSORES)

Referencia absoluta detectada: 48484
Total sensores encontrados en sets: 46

------------------------------------------------------------
Resumen cálculo masivo:
  Total sensores: 46
  Éxitos: 45
  Fallidos: 0
  Self (referencia): 1
------------------------------------------------------------

✅ Exportado: outputs/constantes_calibracion_all_sensors.txt

Muestra de resultados:
 sensor  set  offset_mK  error_mK         method  n_paths
  48060    3  -0.089253  0.000889 weighted_paths        2
  48061    3  -0.123312  0.000883 weighted_paths        2
  48062    3  -0.112787  0.000909 weighted_paths        2
  48063    3  -0.123229  0.000925 weighted_paths        2
  48176    3  -0.135173  0.001101 weighted_paths        2
  48177    3  -0.121985  0.000840 weighted_paths        2
  48202    3  -0.190890  0.001111 weighted_paths        2
  48203    3  -0.163052  0.000832 weighted_paths        2
  48204    3  -0.131497  0.0